# Feature Selection

***** FILL OUT LATER *****

In [1]:
#####
# - IMPORTS
#####
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler

In [2]:
#####
# - LOADING DATASETS & SETTING IT SO THAT ALL COLUMNS SHOW WHEN VIEWING THE DATASETS
#####

batting = pd.read_csv('clean_batting.csv',index_col=0)
pitching = pd.read_csv('clean_pitching.csv',index_col=0)

pd.set_option('display.max_columns', None)

In [3]:
batting

,Name,Age,Tm,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Year,Nxt_BA,Nxt_RBI,Nxt_HR,Nxt_BB,Nxt_SO
0,A.J. Burnett,24,FLA,25,59,50,0,4,1,0,0,0,0,0,1,27,0.080,0.115,0.100,0.215,-42,5,0,1,7,0,0,2001,0.105,3.0,1.0,5.0,28.0
1,A.J. Burnett,25,FLA,29,69,57,1,6,2,0,1,3,0,0,5,28,0.105,0.177,0.193,0.370,-1,11,1,0,7,0,0,2002,0.138,1.0,0.0,0.0,11.0
2,A.J. Burnett,27,FLA,18,37,29,4,4,0,0,0,1,0,0,0,11,0.138,0.138,0.138,0.276,-26,4,1,0,8,0,0,2004,0.147,2.0,1.0,1.0,34.0
3,A.J. Burnett,28,FLA,31,79,68,3,10,2,2,1,2,0,0,1,34,0.147,0.171,0.279,0.451,19,19,1,1,9,0,0,2005,0.063,2.0,0.0,3.0,36.0
4,A.J. Burnett,35,PIT,29,71,63,1,4,0,0,0,2,0,0,3,36,0.063,0.106,0.063,0.170,-51,4,1,0,5,0,0,2012,0.068,2.0,0.0,2.0,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31688,Ángel Pagán#,29,NYM,123,532,478,68,125,24,4,7,56,32,7,44,62,0.262,0.322,0.372,0.694,94,178,4,1,4,5,4,2011,0.288,56.0,8.0,48.0,97.0
31689,Ángel Pagán#,30,SFG,154,659,605,95,174,38,15,8,56,29,7,48,97,0.288,0.338,0.440,0.778,120,266,6,0,2,4,5,2012,0.282,30.0,5.0,23.0,36.0
31690,Ángel Pagán#,31,SFG,71,305,280,44,79,16,3,5,30,9,4,23,36,0.282,0.334,0.414,0.749,113,116,1,0,0,2,0,2013,0.300,27.0,3.0,25.0,53.0
31691,Ángel Pagán#,32,SFG,96,413,383,56,115,21,2,3,27,16,6,25,53,0.300,0.342,0.389,0.731,110,149,5,1,1,3,1,2014,0.262,37.0,3.0,32.0,93.0


In [4]:
pitching

,Name,Age,Tm,W,L,W-L%,ERA,G,GS,GF,CG,SHO,SV,IP,H,R,ER,HR,BB,IBB,SO,HBP,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Year,Nxt_ERA,Nxt_SO,Nxt_WHIP,Nxt_BB,Nxt_W,Nxt_SV
0,A.J. Achter,25,MIN,1,0,1.000,3.27,7,0,1,0,0,0,11.0,14,7,4,2,3,0,5,0,0,0,49,123,5.40,1.545,11.5,1.6,2.5,4.1,1.67,2014,6.75,14.0,1.350,6.0,0.0,0.0
1,A.J. Achter,26,MIN,0,1,0.000,6.75,11,0,4,0,0,0,13.1,12,10,10,4,6,1,14,0,0,0,58,62,6.28,1.350,8.1,2.7,4.1,9.5,2.33,2015,3.11,14.0,1.460,12.0,1.0,0.0
3,A.J. Burnett,22,FLA,4,2,0.667,3.48,7,7,0,0,0,0,41.1,37,23,16,3,25,2,33,0,0,0,182,124,4.30,1.500,8.1,0.7,5.4,7.2,1.32,1999,4.79,57.0,1.500,44.0,3.0,0.0
4,A.J. Burnett,23,FLA,3,7,0.300,4.79,13,13,0,0,0,0,82.2,80,46,44,8,44,3,57,2,0,2,364,92,4.68,1.500,8.7,0.9,4.8,6.2,1.30,2000,4.05,128.0,1.315,83.0,11.0,0.0
5,A.J. Burnett,24,FLA,11,12,0.478,4.05,27,27,0,2,1,0,173.1,145,82,78,20,83,3,128,7,1,7,733,105,4.63,1.315,7.5,1.0,4.3,6.6,1.54,2001,3.30,203.0,1.189,90.0,12.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23609,Zane Smith*,31,PIT,8,8,0.500,3.06,23,22,0,4,3,0,141.0,138,56,48,8,19,3,56,2,0,0,566,114,3.17,1.113,8.8,0.5,1.2,3.6,2.95,1992,4.55,32.0,1.434,22.0,3.0,0.0
23610,Zane Smith*,32,PIT,3,7,0.300,4.55,14,14,0,1,0,0,83.0,97,43,42,5,22,3,32,0,0,2,353,88,3.80,1.434,10.5,0.5,2.4,3.5,1.45,1993,3.27,57.0,1.248,34.0,10.0,0.0
23611,Zane Smith*,33,PIT,10,8,0.556,3.27,25,24,0,2,1,0,157.0,162,67,57,18,34,7,57,0,0,2,645,132,4.55,1.248,9.3,1.0,1.9,3.3,1.68,1994,5.61,47.0,1.509,23.0,8.0,0.0
23612,Zane Smith*,34,BOS,8,8,0.500,5.61,24,21,0,0,0,0,110.2,144,78,69,7,23,1,47,1,1,0,484,87,3.73,1.509,11.7,0.6,1.9,3.8,2.04,1995,5.08,47.0,1.500,21.0,4.0,0.0


In [5]:
#####
# - SETTING UP THE FEATURE SELECTORS
#####

rr = Ridge(alpha=1)

split = TimeSeriesSplit(n_splits=3)

bat_sfs_ba = SequentialFeatureSelector(rr, n_features_to_select=20, cv=split, n_jobs= -1) 
bat_sfs_rbi = SequentialFeatureSelector(rr, n_features_to_select=20, cv=split, n_jobs= -1) 
bat_sfs_hr = SequentialFeatureSelector(rr, n_features_to_select=20, cv=split, n_jobs= -1) 
bat_sfs_bb = SequentialFeatureSelector(rr, n_features_to_select=20, cv=split, n_jobs= -1) 
bat_sfs_so = SequentialFeatureSelector(rr, n_features_to_select=20, cv=split, n_jobs= -1) 

pitch_sfs_era = SequentialFeatureSelector(rr, n_features_to_select=25, cv=split, n_jobs= -1) 
pitch_sfs_so = SequentialFeatureSelector(rr, n_features_to_select=25, cv=split, n_jobs= -1) 
pitch_sfs_whip = SequentialFeatureSelector(rr, n_features_to_select=25, cv=split, n_jobs= -1) 
pitch_sfs_bb = SequentialFeatureSelector(rr, n_features_to_select=25, cv=split, n_jobs= -1) 
pitch_sfs_w = SequentialFeatureSelector(rr, n_features_to_select=25, cv=split, n_jobs= -1) 
pitch_sfs_sv = SequentialFeatureSelector(rr, n_features_to_select=25, cv=split, n_jobs= -1) 

In [6]:
#####
# - REMOVING STRING COLUMNS ALONG WITH COLUMNS TO COMPARE PREDICTIONS TO
#####

remove_bat_cols = ['Name', 'Tm', 'Year', 'Nxt_BA', 'Nxt_RBI', 'Nxt_HR', 'Nxt_BB', 'Nxt_SO']
remove_pitch_cols = ['Name', 'Tm', 'Year', 'Nxt_ERA', 'Nxt_SO', 'Nxt_WHIP', 'Nxt_BB', 'Nxt_W', 'Nxt_SV']

bat_cols = batting.columns[~batting.columns.isin(remove_bat_cols)]
pitch_cols = pitching.columns[~pitching.columns.isin(remove_pitch_cols)]

In [7]:
#####
# - SCALING THE DATA
#####

scaler = MinMaxScaler()
batting.loc[:, bat_cols] = scaler.fit_transform(batting[bat_cols])
pitching.loc[:, pitch_cols] = scaler.fit_transform(pitching[pitch_cols])

In [8]:
batting

,Name,Age,Tm,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Year,Nxt_BA,Nxt_RBI,Nxt_HR,Nxt_BB,Nxt_SO
0,A.J. Burnett,0.206897,FLA,0.067114,0.073454,0.067227,0.000000,0.015267,0.016949,0.000000,0.000000,0.000000,0.000000,0.000000,0.004310,0.121076,0.080,0.115,0.0500,0.080615,0.091627,0.011765,0.000000,0.02,0.175,0.000000,0.000000,2001,0.105,3.0,1.0,5.0,28.0
1,A.J. Burnett,0.241379,FLA,0.093960,0.086340,0.077031,0.006579,0.022901,0.033898,0.000000,0.013699,0.018182,0.000000,0.000000,0.021552,0.125561,0.105,0.177,0.0965,0.138733,0.156398,0.025882,0.027778,0.00,0.175,0.000000,0.000000,2002,0.138,1.0,0.0,0.0,11.0
2,A.J. Burnett,0.310345,FLA,0.020134,0.045103,0.037815,0.026316,0.015267,0.000000,0.000000,0.000000,0.006061,0.000000,0.000000,0.000000,0.049327,0.138,0.138,0.0690,0.103487,0.116904,0.009412,0.027778,0.00,0.200,0.000000,0.000000,2004,0.147,2.0,1.0,1.0,34.0
3,A.J. Burnett,0.344828,FLA,0.107383,0.099227,0.092437,0.019737,0.038168,0.033898,0.086957,0.013699,0.012121,0.000000,0.000000,0.004310,0.152466,0.147,0.171,0.1395,0.169104,0.187994,0.044706,0.027778,0.02,0.225,0.000000,0.000000,2005,0.063,2.0,0.0,3.0,36.0
4,A.J. Burnett,0.586207,PIT,0.093960,0.088918,0.085434,0.006579,0.015267,0.000000,0.000000,0.000000,0.012121,0.000000,0.000000,0.012931,0.161435,0.063,0.106,0.0315,0.063742,0.077409,0.009412,0.027778,0.00,0.125,0.000000,0.000000,2012,0.068,2.0,0.0,2.0,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31688,Ángel Pagán#,0.379310,NYM,0.724832,0.682990,0.666667,0.447368,0.477099,0.406780,0.173913,0.095890,0.339394,0.246154,0.166667,0.189655,0.278027,0.262,0.322,0.1860,0.260217,0.306477,0.418824,0.111111,0.02,0.100,0.277778,0.033333,2011,0.288,56.0,8.0,48.0,97.0
31689,Ángel Pagán#,0.413793,SFG,0.932886,0.846649,0.844538,0.625000,0.664122,0.644068,0.652174,0.109589,0.339394,0.223077,0.166667,0.206897,0.434978,0.288,0.338,0.2200,0.291714,0.347551,0.625882,0.166667,0.00,0.050,0.222222,0.041667,2012,0.282,30.0,5.0,23.0,36.0
31690,Ángel Pagán#,0.448276,SFG,0.375839,0.390464,0.389356,0.289474,0.301527,0.271186,0.130435,0.068493,0.181818,0.069231,0.095238,0.099138,0.161435,0.282,0.334,0.2070,0.280840,0.336493,0.272941,0.027778,0.00,0.000,0.111111,0.000000,2013,0.300,27.0,3.0,25.0,53.0
31691,Ángel Pagán#,0.482759,SFG,0.543624,0.529639,0.533613,0.368421,0.438931,0.355932,0.086957,0.041096,0.163636,0.123077,0.142857,0.107759,0.237668,0.300,0.342,0.1945,0.274091,0.331754,0.350588,0.138889,0.02,0.025,0.166667,0.008333,2014,0.262,37.0,3.0,32.0,93.0


In [9]:
pitching

,Name,Age,Tm,W,L,W-L%,ERA,G,GS,GF,CG,SHO,SV,IP,H,R,ER,HR,BB,IBB,SO,HBP,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Year,Nxt_ERA,Nxt_SO,Nxt_WHIP,Nxt_BB,Nxt_W,Nxt_SV
0,A.J. Achter,0.233333,MIN,0.037037,0.000000,1.000,0.110236,0.019802,0.000000,0.011905,0.000000,0.0,0.0,0.026652,0.034211,0.036364,0.020408,0.04,0.014706,0.000000,0.013055,0.000000,0.0000,0.000000,0.026161,0.077377,0.287988,0.144087,0.303125,0.137931,0.046296,0.220430,0.0668,2014,6.75,14.0,1.350,6.0,0.0,0.0
1,A.J. Achter,0.266667,MIN,0.000000,0.045455,0.000,0.240720,0.059406,0.000000,0.047619,0.000000,0.0,0.0,0.032249,0.028947,0.054545,0.061224,0.08,0.029412,0.041667,0.036554,0.000000,0.0000,0.000000,0.032047,0.032424,0.330439,0.121699,0.196875,0.232759,0.075926,0.510753,0.0932,2015,3.11,14.0,1.460,12.0,1.0,0.0
3,A.J. Burnett,0.133333,FLA,0.148148,0.090909,0.667,0.118110,0.019802,0.142857,0.000000,0.000000,0.0,0.0,0.106876,0.094737,0.133333,0.102041,0.06,0.122549,0.083333,0.086162,0.000000,0.0000,0.000000,0.113146,0.078113,0.234925,0.138921,0.196875,0.060345,0.100000,0.387097,0.0528,1999,4.79,57.0,1.500,44.0,3.0,0.0
4,A.J. Burnett,0.166667,FLA,0.111111,0.318182,0.300,0.167229,0.079208,0.265306,0.000000,0.000000,0.0,0.0,0.216418,0.207895,0.272727,0.292517,0.16,0.215686,0.125000,0.148825,0.095238,0.0000,0.076923,0.232178,0.054532,0.253256,0.138921,0.215625,0.077586,0.088889,0.333333,0.0520,2000,4.05,128.0,1.315,83.0,11.0,0.0
5,A.J. Burnett,0.200000,FLA,0.407407,0.545455,0.478,0.139483,0.217822,0.551020,0.000000,0.066667,0.1,0.0,0.458689,0.378947,0.490909,0.523810,0.40,0.406863,0.125000,0.334204,0.333333,0.0625,0.269231,0.473512,0.064112,0.250844,0.117681,0.178125,0.086207,0.079630,0.354839,0.0616,2001,3.30,203.0,1.189,90.0,12.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23609,Zane Smith*,0.433333,PIT,0.296296,0.363636,0.500,0.102362,0.178218,0.448980,0.000000,0.133333,0.3,0.0,0.373134,0.360526,0.333333,0.319728,0.16,0.093137,0.125000,0.146214,0.095238,0.0000,0.000000,0.364290,0.070744,0.180415,0.094489,0.218750,0.043103,0.022222,0.193548,0.1180,1992,4.55,32.0,1.434,22.0,3.0,0.0
23610,Zane Smith*,0.466667,PIT,0.111111,0.318182,0.300,0.158230,0.089109,0.285714,0.000000,0.033333,0.0,0.0,0.218550,0.252632,0.254545,0.278912,0.10,0.107843,0.125000,0.083551,0.000000,0.0000,0.076923,0.224984,0.051584,0.210806,0.131343,0.271875,0.043103,0.044444,0.188172,0.0580,1993,3.27,57.0,1.248,34.0,10.0,0.0
23611,Zane Smith*,0.500000,PIT,0.370370,0.363636,0.556,0.110236,0.198020,0.489796,0.000000,0.066667,0.1,0.0,0.415778,0.423684,0.400000,0.380952,0.36,0.166667,0.291667,0.148825,0.000000,0.0000,0.076923,0.415958,0.084009,0.246985,0.109989,0.234375,0.086207,0.035185,0.177419,0.0672,1994,5.61,47.0,1.509,23.0,8.0,0.0
23612,Zane Smith*,0.533333,BOS,0.296296,0.363636,0.500,0.197975,0.188119,0.428571,0.000000,0.000000,0.0,0.0,0.291045,0.376316,0.466667,0.462585,0.14,0.112745,0.041667,0.122715,0.047619,0.0625,0.000000,0.310661,0.050847,0.207429,0.139954,0.309375,0.051724,0.035185,0.204301,0.0816,1995,5.08,47.0,1.500,21.0,4.0,0.0


In [10]:
#####
# - FITTING EACH OF THE PREDICTORS
#####

bat_sfs_ba.fit(batting[bat_cols], batting['Nxt_BA'])
bat_sfs_rbi.fit(batting[bat_cols], batting['Nxt_RBI'])
bat_sfs_hr.fit(batting[bat_cols], batting['Nxt_HR'])
bat_sfs_bb.fit(batting[bat_cols], batting['Nxt_BB'])
bat_sfs_so.fit(batting[bat_cols], batting['Nxt_SO'])

pitch_sfs_era.fit(pitching[pitch_cols], pitching['Nxt_ERA'])
pitch_sfs_so.fit(pitching[pitch_cols], pitching['Nxt_SO'])
pitch_sfs_whip.fit(pitching[pitch_cols], pitching['Nxt_WHIP'])
pitch_sfs_bb.fit(pitching[pitch_cols], pitching['Nxt_BB'])
pitch_sfs_w.fit(pitching[pitch_cols], pitching['Nxt_W'])
pitch_sfs_sv.fit(pitching[pitch_cols], pitching['Nxt_SV'])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=Ridge(alpha=1), n_features_to_select=25,
                          n_jobs=-1)

In [11]:
#####
# - SETTING UP LIST OF PREDICTORS FOR EACH FEATURE
#####

bat_ba_preds = list(bat_cols[bat_sfs_ba.get_support()])
bat_rbi_preds = list(bat_cols[bat_sfs_rbi.get_support()])
bat_hr_preds = list(bat_cols[bat_sfs_hr.get_support()])
bat_bb_preds = list(bat_cols[bat_sfs_bb.get_support()])
bat_so_preds = list(bat_cols[bat_sfs_so.get_support()])

pitch_era_preds = list(pitch_cols[pitch_sfs_era.get_support()])
pitch_so_preds = list(pitch_cols[pitch_sfs_so.get_support()])
pitch_whip_preds = list(pitch_cols[pitch_sfs_whip.get_support()])
pitch_bb_preds = list(pitch_cols[pitch_sfs_bb.get_support()])
pitch_w_preds = list(pitch_cols[pitch_sfs_w.get_support()])
pitch_sv_preds = list(pitch_cols[pitch_sfs_sv.get_support()])

In [12]:
bat_ba_preds

['Age',
 'G',
 'PA',
 'AB',
 'R',
 'H',
 '2B',
 '3B',
 'HR',
 'RBI',
 'CS',
 'BB',
 'SO',
 'OBP',
 'OPS',
 'TB',
 'GDP',
 'SH',
 'SF',
 'IBB']

In [13]:
bat_rbi_preds

['Age',
 'G',
 'PA',
 'AB',
 'H',
 '2B',
 '3B',
 'RBI',
 'SO',
 'BA',
 'OBP',
 'SLG',
 'OPS',
 'OPS+',
 'TB',
 'GDP',
 'HBP',
 'SH',
 'SF',
 'IBB']

In [14]:
bat_hr_preds

['Age',
 'G',
 'PA',
 'AB',
 'R',
 'H',
 '2B',
 '3B',
 'HR',
 'RBI',
 'BB',
 'SO',
 'BA',
 'OBP',
 'SLG',
 'OPS',
 'OPS+',
 'GDP',
 'SH',
 'IBB']

In [15]:
bat_bb_preds

['Age',
 'G',
 'AB',
 'R',
 'H',
 '2B',
 '3B',
 'RBI',
 'SB',
 'BB',
 'SO',
 'BA',
 'OBP',
 'SLG',
 'OPS',
 'OPS+',
 'HBP',
 'SH',
 'SF',
 'IBB']

In [16]:
bat_so_preds

['Age',
 'G',
 'PA',
 'AB',
 'R',
 'H',
 '2B',
 '3B',
 'RBI',
 'SB',
 'SO',
 'BA',
 'OBP',
 'SLG',
 'OPS',
 'OPS+',
 'GDP',
 'SH',
 'SF',
 'IBB']

In [17]:
pitch_era_preds

['Age',
 'W',
 'L',
 'GS',
 'GF',
 'CG',
 'SHO',
 'SV',
 'IP',
 'H',
 'R',
 'ER',
 'HR',
 'BB',
 'IBB',
 'SO',
 'HBP',
 'BK',
 'WP',
 'BF',
 'FIP',
 'WHIP',
 'HR9',
 'BB9',
 'SO9']

In [18]:
pitch_so_preds

['Age',
 'W',
 'L',
 'W-L%',
 'ERA',
 'G',
 'GS',
 'GF',
 'CG',
 'SHO',
 'SV',
 'IP',
 'HR',
 'SO',
 'HBP',
 'BK',
 'WP',
 'BF',
 'ERA+',
 'FIP',
 'WHIP',
 'HR9',
 'BB9',
 'SO9',
 'SO/W']

In [19]:
pitch_whip_preds

['Age',
 'W',
 'L',
 'W-L%',
 'G',
 'CG',
 'SHO',
 'SV',
 'IP',
 'H',
 'R',
 'ER',
 'HR',
 'BB',
 'IBB',
 'SO',
 'HBP',
 'BK',
 'WP',
 'BF',
 'ERA+',
 'HR9',
 'BB9',
 'SO9',
 'SO/W']

In [20]:
pitch_bb_preds

['Age',
 'W',
 'W-L%',
 'ERA',
 'G',
 'GF',
 'CG',
 'SHO',
 'SV',
 'IP',
 'R',
 'ER',
 'HR',
 'BB',
 'IBB',
 'SO',
 'HBP',
 'WP',
 'ERA+',
 'FIP',
 'WHIP',
 'HR9',
 'BB9',
 'SO9',
 'SO/W']

In [21]:
pitch_w_preds

['Age',
 'W',
 'L',
 'W-L%',
 'ERA',
 'G',
 'GS',
 'GF',
 'CG',
 'SV',
 'IP',
 'R',
 'ER',
 'HR',
 'BB',
 'SO',
 'BK',
 'WP',
 'BF',
 'ERA+',
 'FIP',
 'HR9',
 'BB9',
 'SO9',
 'SO/W']

In [22]:
pitch_sv_preds

['Age',
 'L',
 'W-L%',
 'ERA',
 'GF',
 'CG',
 'SHO',
 'SV',
 'IP',
 'H',
 'R',
 'ER',
 'HR',
 'SO',
 'HBP',
 'BK',
 'BF',
 'ERA+',
 'FIP',
 'WHIP',
 'H9',
 'HR9',
 'BB9',
 'SO9',
 'SO/W']

# Decision Tree Model

In [23]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error as mse

dtr = DecisionTreeRegressor(splitter ='best',max_depth=2,min_samples_split=17)
dtr2 = DecisionTreeRegressor(splitter ='best',max_depth=3,min_samples_split=17)
dtr3 = DecisionTreeRegressor(splitter ='best',max_depth=4,min_samples_split=17)
dtr4 = DecisionTreeRegressor(splitter ='best',max_depth=5,min_samples_split=17)
dtr5 = DecisionTreeRegressor(splitter ='best',max_depth=6,min_samples_split=17)

dtr6 = DecisionTreeRegressor(splitter ='best',max_depth=7,min_samples_split=17)
dtr7 = DecisionTreeRegressor(splitter ='best',max_depth=8,min_samples_split=17)
dtr8 = DecisionTreeRegressor(splitter ='best',max_depth=9,min_samples_split=17)
dtr9 = DecisionTreeRegressor(splitter ='best',max_depth=10,min_samples_split=17)
dtr10 = DecisionTreeRegressor(splitter ='best',max_depth=11,min_samples_split=17)

## Batters

#### Batting Average

In [24]:
#####
# - FUNCTION TO SPLIT BETWEEN TRAINING AND TEST SETS AND TO RUN THROUGH THE MODEL
#####

def bat_ba_model(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    years = sorted(data["Year"].unique())
    
    for i in range(start, len(years), step):
        current_year = years[i]
        train = data[data["Year"] < current_year]
        test = data[data["Year"] == current_year]
                
        model.fit(train[predictors], train["Nxt_BA"])
        
        preds = model.predict(test[predictors])

        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["Nxt_BA"], preds], axis=1)
        combined.columns = ["actual", "predicted"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [25]:
predicted = bat_ba_model(batting, dtr, bat_ba_preds)
predicted2 = bat_ba_model(batting, dtr2, bat_ba_preds)
predicted3 = bat_ba_model(batting, dtr3, bat_ba_preds)
predicted4 = bat_ba_model(batting, dtr4, bat_ba_preds)
predicted5 = bat_ba_model(batting, dtr5, bat_ba_preds)

predicted6 = bat_ba_model(batting, dtr6, bat_ba_preds)
predicted7 = bat_ba_model(batting, dtr7, bat_ba_preds)
predicted8 = bat_ba_model(batting, dtr8, bat_ba_preds)
predicted9 = bat_ba_model(batting, dtr9, bat_ba_preds)
predicted10 = bat_ba_model(batting, dtr10, bat_ba_preds)

In [26]:
print('Decision Tree BA 01 MSE:',round(mse(predicted["actual"], predicted["predicted"]), 5))
print('Decision Tree BA 02 MSE:',round(mse(predicted2["actual"], predicted2["predicted"]), 5))
print('Decision Tree BA 03 MSE:',round(mse(predicted3["actual"], predicted3["predicted"]), 5))
print('Decision Tree BA 04 MSE:',round(mse(predicted4["actual"], predicted4["predicted"]), 5))
print('Decision Tree BA 05 MSE:',round(mse(predicted5["actual"], predicted5["predicted"]), 5))
print('Decision Tree BA 06 MSE:',round(mse(predicted6["actual"], predicted6["predicted"]), 5))
print('Decision Tree BA 07 MSE:',round(mse(predicted7["actual"], predicted7["predicted"]), 5))
print('Decision Tree BA 08 MSE:',round(mse(predicted8["actual"], predicted8["predicted"]), 5))
print('Decision Tree BA 09 MSE:',round(mse(predicted9["actual"], predicted9["predicted"]), 5))
print('Decision Tree BA 10 MSE:',round(mse(predicted10["actual"], predicted10["predicted"]), 5))

Decision Tree BA 01 MSE: 0.00394
Decision Tree BA 02 MSE: 0.00385
Decision Tree BA 03 MSE: 0.00379
Decision Tree BA 04 MSE: 0.00381
Decision Tree BA 05 MSE: 0.00392
Decision Tree BA 06 MSE: 0.00401
Decision Tree BA 07 MSE: 0.00412
Decision Tree BA 08 MSE: 0.00424
Decision Tree BA 09 MSE: 0.00438
Decision Tree BA 10 MSE: 0.00453


In [27]:
bat_ba_dtreg = round(mse(predicted3["actual"], predicted3["predicted"]), 5)
batting = batting.join(predicted3, how='outer').fillna(0)
batting = batting.drop('actual', axis=1)
batting.rename(columns = {'predicted':'BA_DTR'}, inplace = True)

#### RBI

In [28]:
#####
# - FUNCTION TO SPLIT BETWEEN TRAINING AND TEST SETS AND TO RUN THROUGH THE MODEL
#####
def bat_rbi_model(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    years = sorted(data["Year"].unique())
    
    for i in range(start, len(years), step):
        current_year = years[i]
        train = data[data["Year"] < current_year]
        test = data[data["Year"] == current_year]
                
        model.fit(train[predictors], train["Nxt_RBI"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["Nxt_RBI"], preds], axis=1)
        combined.columns = ["actual", "predicted"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [29]:
predicted = bat_rbi_model(batting, dtr, bat_rbi_preds)
predicted2 = bat_rbi_model(batting, dtr2, bat_rbi_preds)
predicted3 = bat_rbi_model(batting, dtr3, bat_rbi_preds)
predicted4 = bat_rbi_model(batting, dtr4, bat_rbi_preds)
predicted5 = bat_rbi_model(batting, dtr5, bat_rbi_preds)

predicted6 = bat_rbi_model(batting, dtr6, bat_rbi_preds)
predicted7 = bat_rbi_model(batting, dtr7, bat_rbi_preds)
predicted8 = bat_rbi_model(batting, dtr8, bat_rbi_preds)
predicted9 = bat_rbi_model(batting, dtr9, bat_rbi_preds)
predicted10 = bat_rbi_model(batting, dtr10, bat_rbi_preds)

In [30]:
print('Decision Tree RBI 01 MSE:',round(mse(predicted["actual"], predicted["predicted"]), 5))
print('Decision Tree RBI 02 MSE:',round(mse(predicted2["actual"], predicted2["predicted"]), 5))
print('Decision Tree RBI 03 MSE:',round(mse(predicted3["actual"], predicted3["predicted"]), 5))
print('Decision Tree RBI 04 MSE:',round(mse(predicted4["actual"], predicted4["predicted"]), 5))
print('Decision Tree RBI 05 MSE:',round(mse(predicted5["actual"], predicted5["predicted"]), 5))
print('Decision Tree RBI 06 MSE:',round(mse(predicted6["actual"], predicted6["predicted"]), 5))
print('Decision Tree RBI 07 MSE:',round(mse(predicted7["actual"], predicted7["predicted"]), 5))
print('Decision Tree RBI 08 MSE:',round(mse(predicted8["actual"], predicted8["predicted"]), 5))
print('Decision Tree RBI 09 MSE:',round(mse(predicted9["actual"], predicted9["predicted"]), 5))
print('Decision Tree RBI 10 MSE:',round(mse(predicted10["actual"], predicted10["predicted"]), 5))

Decision Tree RBI 01 MSE: 472.33723
Decision Tree RBI 02 MSE: 443.68285
Decision Tree RBI 03 MSE: 431.89804
Decision Tree RBI 04 MSE: 427.9941
Decision Tree RBI 05 MSE: 429.20839
Decision Tree RBI 06 MSE: 438.29464
Decision Tree RBI 07 MSE: 454.60942
Decision Tree RBI 08 MSE: 477.48259
Decision Tree RBI 09 MSE: 499.13223
Decision Tree RBI 10 MSE: 518.51696


In [31]:
bat_rbi_dtreg = round(mse(predicted4["actual"], predicted4["predicted"]), 5)
batting = batting.join(predicted4, how='outer').fillna(0)
batting = batting.drop('actual', axis=1)
batting.rename(columns = {'predicted':'RBI_DTR'}, inplace = True)

#### Home Runs

In [32]:
#####
# - FUNCTION TO SPLIT BETWEEN TRAINING AND TEST SETS AND TO RUN THROUGH THE MODEL
#####
def bat_hr_model(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    years = sorted(data["Year"].unique())
    
    for i in range(start, len(years), step):
        current_year = years[i]
        train = data[data["Year"] < current_year]
        test = data[data["Year"] == current_year]
                
        model.fit(train[predictors], train["Nxt_HR"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["Nxt_HR"], preds], axis=1)
        combined.columns = ["actual", "predicted"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [33]:
predicted = bat_hr_model(batting, dtr, bat_hr_preds)
predicted2 = bat_hr_model(batting, dtr2, bat_hr_preds)
predicted3 = bat_hr_model(batting, dtr3, bat_hr_preds)
predicted4 = bat_hr_model(batting, dtr4, bat_hr_preds)
predicted5 = bat_hr_model(batting, dtr5, bat_hr_preds)

predicted6 = bat_hr_model(batting, dtr6, bat_hr_preds)
predicted7 = bat_hr_model(batting, dtr7, bat_hr_preds)
predicted8 = bat_hr_model(batting, dtr8, bat_hr_preds)
predicted9 = bat_hr_model(batting, dtr9, bat_hr_preds)
predicted10 = bat_hr_model(batting, dtr10, bat_hr_preds)

In [34]:
print('Decision Tree HR 01 MSE:',round(mse(predicted["actual"], predicted["predicted"]), 5))
print('Decision Tree HR 02 MSE:',round(mse(predicted2["actual"], predicted2["predicted"]), 5))
print('Decision Tree HR 03 MSE:',round(mse(predicted3["actual"], predicted3["predicted"]), 5))
print('Decision Tree HR 04 MSE:',round(mse(predicted4["actual"], predicted4["predicted"]), 5))
print('Decision Tree HR 05 MSE:',round(mse(predicted5["actual"], predicted5["predicted"]), 5))
print('Decision Tree HR 06 MSE:',round(mse(predicted6["actual"], predicted6["predicted"]), 5))
print('Decision Tree HR 07 MSE:',round(mse(predicted7["actual"], predicted7["predicted"]), 5))
print('Decision Tree HR 08 MSE:',round(mse(predicted8["actual"], predicted8["predicted"]), 5))
print('Decision Tree HR 09 MSE:',round(mse(predicted9["actual"], predicted9["predicted"]), 5))
print('Decision Tree HR 10 MSE:',round(mse(predicted10["actual"], predicted10["predicted"]), 5))

Decision Tree HR 01 MSE: 45.40527
Decision Tree HR 02 MSE: 42.47999
Decision Tree HR 03 MSE: 41.79824
Decision Tree HR 04 MSE: 41.95369
Decision Tree HR 05 MSE: 42.5082
Decision Tree HR 06 MSE: 44.14754
Decision Tree HR 07 MSE: 45.70941
Decision Tree HR 08 MSE: 47.6102
Decision Tree HR 09 MSE: 49.43648
Decision Tree HR 10 MSE: 51.59455


In [35]:
bat_hr_dtreg = round(mse(predicted3["actual"], predicted3["predicted"]), 5)
batting = batting.join(predicted3, how='outer').fillna(0)
batting = batting.drop('actual', axis=1)
batting.rename(columns = {'predicted':'HR_DTR'}, inplace = True)

#### Walks

In [36]:
#####
# - FUNCTION TO SPLIT BETWEEN TRAINING AND TEST SETS AND TO RUN THROUGH THE MODEL
#####
def bat_bb_model(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    years = sorted(data["Year"].unique())
    
    for i in range(start, len(years), step):
        current_year = years[i]
        train = data[data["Year"] < current_year]
        test = data[data["Year"] == current_year]
                
        model.fit(train[predictors], train["Nxt_BB"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["Nxt_BB"], preds], axis=1)
        combined.columns = ["actual", "predicted"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [37]:
predicted = bat_bb_model(batting, dtr, bat_bb_preds)
predicted2 = bat_bb_model(batting, dtr2, bat_bb_preds)
predicted3 = bat_bb_model(batting, dtr3, bat_bb_preds)
predicted4 = bat_bb_model(batting, dtr4, bat_bb_preds)
predicted5 = bat_bb_model(batting, dtr5, bat_bb_preds)

predicted6 = bat_bb_model(batting, dtr6, bat_bb_preds)
predicted7 = bat_bb_model(batting, dtr7, bat_bb_preds)
predicted8 = bat_bb_model(batting, dtr8, bat_bb_preds)
predicted9 = bat_bb_model(batting, dtr9, bat_bb_preds)
predicted10 = bat_bb_model(batting, dtr10, bat_bb_preds)

In [38]:
print('Decision Tree BB 01 MSE:',round(mse(predicted["actual"], predicted["predicted"]), 5))
print('Decision Tree BB 02 MSE:',round(mse(predicted2["actual"], predicted2["predicted"]), 5))
print('Decision Tree BB 03 MSE:',round(mse(predicted3["actual"], predicted3["predicted"]), 5))
print('Decision Tree BB 04 MSE:',round(mse(predicted4["actual"], predicted4["predicted"]), 5))
print('Decision Tree BB 05 MSE:',round(mse(predicted5["actual"], predicted5["predicted"]), 5))
print('Decision Tree BB 06 MSE:',round(mse(predicted6["actual"], predicted6["predicted"]), 5))
print('Decision Tree BB 07 MSE:',round(mse(predicted7["actual"], predicted7["predicted"]), 5))
print('Decision Tree BB 08 MSE:',round(mse(predicted8["actual"], predicted8["predicted"]), 5))
print('Decision Tree BB 09 MSE:',round(mse(predicted9["actual"], predicted9["predicted"]), 5))
print('Decision Tree BB 10 MSE:',round(mse(predicted10["actual"], predicted10["predicted"]), 5))

Decision Tree BB 01 MSE: 286.69695
Decision Tree BB 02 MSE: 259.98251
Decision Tree BB 03 MSE: 251.93732
Decision Tree BB 04 MSE: 248.42436
Decision Tree BB 05 MSE: 252.84353
Decision Tree BB 06 MSE: 259.16713
Decision Tree BB 07 MSE: 267.90633
Decision Tree BB 08 MSE: 282.54411
Decision Tree BB 09 MSE: 295.05162
Decision Tree BB 10 MSE: 305.78709


In [39]:
bat_bb_dtreg = round(mse(predicted4["actual"], predicted4["predicted"]), 5)
batting = batting.join(predicted4, how='outer').fillna(0)
batting = batting.drop('actual', axis=1)
batting.rename(columns = {'predicted':'BB_DTR'}, inplace = True)

#### Strikeouts

In [40]:
#####
# - FUNCTION TO SPLIT BETWEEN TRAINING AND TEST SETS AND TO RUN THROUGH THE MODEL
#####
def bat_so_model(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    years = sorted(data["Year"].unique())
    
    for i in range(start, len(years), step):
        current_year = years[i]
        train = data[data["Year"] < current_year]
        test = data[data["Year"] == current_year]
                
        model.fit(train[predictors], train["Nxt_SO"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["Nxt_SO"], preds], axis=1)
        combined.columns = ["actual", "predicted"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [41]:
predicted = bat_so_model(batting, dtr, bat_so_preds)
predicted2 = bat_so_model(batting, dtr2, bat_so_preds)
predicted3 = bat_so_model(batting, dtr3, bat_so_preds)
predicted4 = bat_so_model(batting, dtr4, bat_so_preds)
predicted5 = bat_so_model(batting, dtr5, bat_so_preds)

predicted6 = bat_so_model(batting, dtr6, bat_so_preds)
predicted7 = bat_so_model(batting, dtr7, bat_so_preds)
predicted8 = bat_so_model(batting, dtr8, bat_so_preds)
predicted9 = bat_so_model(batting, dtr9, bat_so_preds)
predicted10 = bat_so_model(batting, dtr10, bat_so_preds)

In [42]:
print('Decision Tree SO 01 MSE:',round(mse(predicted["actual"], predicted["predicted"]), 5))
print('Decision Tree SO 02 MSE:',round(mse(predicted2["actual"], predicted2["predicted"]), 5))
print('Decision Tree SO 03 MSE:',round(mse(predicted3["actual"], predicted3["predicted"]), 5))
print('Decision Tree SO 04 MSE:',round(mse(predicted4["actual"], predicted4["predicted"]), 5))
print('Decision Tree SO 05 MSE:',round(mse(predicted5["actual"], predicted5["predicted"]), 5))
print('Decision Tree SO 06 MSE:',round(mse(predicted6["actual"], predicted6["predicted"]), 5))
print('Decision Tree SO 07 MSE:',round(mse(predicted7["actual"], predicted7["predicted"]), 5))
print('Decision Tree SO 08 MSE:',round(mse(predicted8["actual"], predicted8["predicted"]), 5))
print('Decision Tree SO 09 MSE:',round(mse(predicted9["actual"], predicted9["predicted"]), 5))
print('Decision Tree SO 10 MSE:',round(mse(predicted10["actual"], predicted10["predicted"]), 5))

Decision Tree SO 01 MSE: 809.00067
Decision Tree SO 02 MSE: 753.4624
Decision Tree SO 03 MSE: 724.17205
Decision Tree SO 04 MSE: 707.86623
Decision Tree SO 05 MSE: 712.72741
Decision Tree SO 06 MSE: 721.13206
Decision Tree SO 07 MSE: 745.57872
Decision Tree SO 08 MSE: 770.96902
Decision Tree SO 09 MSE: 799.95276
Decision Tree SO 10 MSE: 836.0809


In [43]:
bat_so_dtreg = round(mse(predicted4["actual"], predicted4["predicted"]), 5)
batting = batting.join(predicted4, how='outer').fillna(0)
batting = batting.drop('actual', axis=1)
batting.rename(columns = {'predicted':'SO_DTR'}, inplace = True)

In [44]:
batting

,Name,Age,Tm,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Year,Nxt_BA,Nxt_RBI,Nxt_HR,Nxt_BB,Nxt_SO,BA_DTR,RBI_DTR,HR_DTR,BB_DTR,SO_DTR
0,A.J. Burnett,0.206897,FLA,0.067114,0.073454,0.067227,0.000000,0.015267,0.016949,0.000000,0.000000,0.000000,0.000000,0.000000,0.004310,0.121076,0.080,0.115,0.0500,0.080615,0.091627,0.011765,0.000000,0.02,0.175,0.000000,0.000000,2001,0.105,3.0,1.0,5.0,28.0,0.135482,2.833876,0.447870,2.164090,18.911997
1,A.J. Burnett,0.241379,FLA,0.093960,0.086340,0.077031,0.006579,0.022901,0.033898,0.000000,0.013699,0.018182,0.000000,0.000000,0.021552,0.125561,0.105,0.177,0.0965,0.138733,0.156398,0.025882,0.027778,0.00,0.175,0.000000,0.000000,2002,0.138,1.0,0.0,0.0,11.0,0.132434,5.388848,2.772621,4.559322,17.494403
2,A.J. Burnett,0.310345,FLA,0.020134,0.045103,0.037815,0.026316,0.015267,0.000000,0.000000,0.000000,0.006061,0.000000,0.000000,0.000000,0.049327,0.138,0.138,0.0690,0.103487,0.116904,0.009412,0.027778,0.00,0.200,0.000000,0.000000,2004,0.147,2.0,1.0,1.0,34.0,0.142344,5.869801,0.466667,3.109235,17.404954
3,A.J. Burnett,0.344828,FLA,0.107383,0.099227,0.092437,0.019737,0.038168,0.033898,0.086957,0.013699,0.012121,0.000000,0.000000,0.004310,0.152466,0.147,0.171,0.1395,0.169104,0.187994,0.044706,0.027778,0.02,0.225,0.000000,0.000000,2005,0.063,2.0,0.0,3.0,36.0,0.189947,5.660428,2.798278,3.109510,24.019139
4,A.J. Burnett,0.586207,PIT,0.093960,0.088918,0.085434,0.006579,0.015267,0.000000,0.000000,0.000000,0.012121,0.000000,0.000000,0.012931,0.161435,0.063,0.106,0.0315,0.063742,0.077409,0.009412,0.027778,0.00,0.125,0.000000,0.000000,2012,0.068,2.0,0.0,2.0,33.0,0.138198,3.314429,0.572555,1.318576,20.037579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31688,Ángel Pagán#,0.379310,NYM,0.724832,0.682990,0.666667,0.447368,0.477099,0.406780,0.173913,0.095890,0.339394,0.246154,0.166667,0.189655,0.278027,0.262,0.322,0.1860,0.260217,0.306477,0.418824,0.111111,0.02,0.100,0.277778,0.033333,2011,0.288,56.0,8.0,48.0,97.0,0.255850,44.532234,7.549062,35.769741,59.097541
31689,Ángel Pagán#,0.413793,SFG,0.932886,0.846649,0.844538,0.625000,0.664122,0.644068,0.652174,0.109589,0.339394,0.223077,0.166667,0.206897,0.434978,0.288,0.338,0.2200,0.291714,0.347551,0.625882,0.166667,0.00,0.050,0.222222,0.041667,2012,0.282,30.0,5.0,23.0,36.0,0.281584,44.508824,7.590038,41.824375,85.551304
31690,Ángel Pagán#,0.448276,SFG,0.375839,0.390464,0.389356,0.289474,0.301527,0.271186,0.130435,0.068493,0.181818,0.069231,0.095238,0.099138,0.161435,0.282,0.334,0.2070,0.280840,0.336493,0.272941,0.027778,0.00,0.000,0.111111,0.000000,2013,0.300,27.0,3.0,25.0,53.0,0.251485,30.791922,6.812576,23.675464,34.509407
31691,Ángel Pagán#,0.482759,SFG,0.543624,0.529639,0.533613,0.368421,0.438931,0.355932,0.086957,0.041096,0.163636,0.123077,0.142857,0.107759,0.237668,0.300,0.342,0.1945,0.274091,0.331754,0.350588,0.138889,0.02,0.025,0.166667,0.008333,2014,0.262,37.0,3.0,32.0,93.0,0.269237,31.030322,4.487515,23.834653,37.835928


In [45]:
batting[['Nxt_RBI','Nxt_HR','Nxt_BB','Nxt_SO']].describe()

,Nxt_RBI,Nxt_HR,Nxt_BB,Nxt_SO
count,26768.000000,26768.000000,26768.000000,26768.000000
mean,33.324866,7.594217,25.760759,48.400702
std,31.153845,9.408168,24.697009,38.108902
min,0.000000,0.000000,0.000000,0.000000
25%,6.000000,0.000000,5.000000,18.000000
50%,25.000000,4.000000,19.000000,39.000000
75%,53.000000,12.000000,40.000000,72.000000
max,165.000000,73.000000,232.000000,223.000000


In [46]:
#####
# - UNSCALING DATA (INVERSE TRANSFORM CAUSED ERRORS)
#####

## - BA
batting[['BA_DTR']] = round(batting[['BA_DTR']], 3)

## - RBI
bat_rbi_dtr = batting[['RBI_DTR']]
#scaled from 0 to 165
min = 0
max = 165
bat_rbi_dtr * min + (max - min)

batting['RBI_DTR'] = bat_rbi_dtr.astype(int)


## - HR
bat_hr_dtr = batting[['HR_DTR']]
#scaled from 0 to 73
min = 0
max = 73
bat_hr_dtr * min + (max - min)

batting['HR_DTR'] = bat_hr_dtr.astype(int)

## - BB
bat_bb_dtr = batting[['BB_DTR']]
#scaled from 0 to 232
min = 0
max = 232
bat_bb_dtr * min + (max - min)

batting['BB_DTR'] = bat_bb_dtr.astype(int)

## - SO
bat_so_dtr = batting[['SO_DTR']]
#scaled from 0 to 223
min = 0
max = 223
bat_so_dtr * min + (max - min)

batting['SO_DTR'] = bat_so_dtr.astype(int)

batting.head()

,Name,Age,Tm,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Year,Nxt_BA,Nxt_RBI,Nxt_HR,Nxt_BB,Nxt_SO,BA_DTR,RBI_DTR,HR_DTR,BB_DTR,SO_DTR
0,A.J. Burnett,0.206897,FLA,0.067114,0.073454,0.067227,0.000000,0.015267,0.016949,0.000000,0.000000,0.000000,0.0,0.0,0.004310,0.121076,0.080,0.115,0.0500,0.080615,0.091627,0.011765,0.000000,0.02,0.175,0.0,0.0,2001,0.105,3.0,1.0,5.0,28.0,0.135,2,0,2,18
1,A.J. Burnett,0.241379,FLA,0.093960,0.086340,0.077031,0.006579,0.022901,0.033898,0.000000,0.013699,0.018182,0.0,0.0,0.021552,0.125561,0.105,0.177,0.0965,0.138733,0.156398,0.025882,0.027778,0.00,0.175,0.0,0.0,2002,0.138,1.0,0.0,0.0,11.0,0.132,5,2,4,17
2,A.J. Burnett,0.310345,FLA,0.020134,0.045103,0.037815,0.026316,0.015267,0.000000,0.000000,0.000000,0.006061,0.0,0.0,0.000000,0.049327,0.138,0.138,0.0690,0.103487,0.116904,0.009412,0.027778,0.00,0.200,0.0,0.0,2004,0.147,2.0,1.0,1.0,34.0,0.142,5,0,3,17
3,A.J. Burnett,0.344828,FLA,0.107383,0.099227,0.092437,0.019737,0.038168,0.033898,0.086957,0.013699,0.012121,0.0,0.0,0.004310,0.152466,0.147,0.171,0.1395,0.169104,0.187994,0.044706,0.027778,0.02,0.225,0.0,0.0,2005,0.063,2.0,0.0,3.0,36.0,0.190,5,2,3,24
4,A.J. Burnett,0.586207,PIT,0.093960,0.088918,0.085434,0.006579,0.015267,0.000000,0.000000,0.000000,0.012121,0.0,0.0,0.012931,0.161435,0.063,0.106,0.0315,0.063742,0.077409,0.009412,0.027778,0.00,0.125,0.0,0.0,2012,0.068,2.0,0.0,2.0,33.0,0.138,3,0,1,20


## Pitchers

#### ERA

In [47]:
#####
# - FUNCTION TO SPLIT BETWEEN TRAINING AND TEST SETS AND TO RUN THROUGH THE MODEL
#####
def pitch_era_model(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    years = sorted(data["Year"].unique())
    
    for i in range(start, len(years), step):
        current_year = years[i]
        train = data[data["Year"] < current_year]
        test = data[data["Year"] == current_year]
                
        model.fit(train[predictors], train["Nxt_ERA"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["Nxt_ERA"], preds], axis=1)
        combined.columns = ["actual", "predicted"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [48]:
predicted = pitch_era_model(pitching, dtr, pitch_era_preds)
predicted2 = pitch_era_model(pitching, dtr2, pitch_era_preds)
predicted3 = pitch_era_model(pitching, dtr3, pitch_era_preds)
predicted4 = pitch_era_model(pitching, dtr4, pitch_era_preds)
predicted5 = pitch_era_model(pitching, dtr5, pitch_era_preds)

predicted6 = pitch_era_model(pitching, dtr6, pitch_era_preds)
predicted7 = pitch_era_model(pitching, dtr7, pitch_era_preds)
predicted8 = pitch_era_model(pitching, dtr8, pitch_era_preds)
predicted9 = pitch_era_model(pitching, dtr9, pitch_era_preds)
predicted10 = pitch_era_model(pitching, dtr10, pitch_era_preds)

In [49]:
print('Decision Tree ERA 01 MSE:',round(mse(predicted["actual"], predicted["predicted"]), 5))
print('Decision Tree ERA 02 MSE:',round(mse(predicted2["actual"], predicted2["predicted"]), 5))
print('Decision Tree ERA 03 MSE:',round(mse(predicted3["actual"], predicted3["predicted"]), 5))
print('Decision Tree ERA 04 MSE:',round(mse(predicted4["actual"], predicted4["predicted"]), 5))
print('Decision Tree ERA 05 MSE:',round(mse(predicted5["actual"], predicted5["predicted"]), 5))
print('Decision Tree ERA 06 MSE:',round(mse(predicted6["actual"], predicted6["predicted"]), 5))
print('Decision Tree ERA 07 MSE:',round(mse(predicted7["actual"], predicted7["predicted"]), 5))
print('Decision Tree ERA 08 MSE:',round(mse(predicted8["actual"], predicted8["predicted"]), 5))
print('Decision Tree ERA 09 MSE:',round(mse(predicted9["actual"], predicted9["predicted"]), 5))
print('Decision Tree ERA 10 MSE:',round(mse(predicted10["actual"], predicted10["predicted"]), 5))

Decision Tree ERA 01 MSE: 3.64208
Decision Tree ERA 02 MSE: 3.6554
Decision Tree ERA 03 MSE: 3.70495
Decision Tree ERA 04 MSE: 3.75198
Decision Tree ERA 05 MSE: 3.85684
Decision Tree ERA 06 MSE: 3.93114
Decision Tree ERA 07 MSE: 4.03197
Decision Tree ERA 08 MSE: 4.13758
Decision Tree ERA 09 MSE: 4.30984
Decision Tree ERA 10 MSE: 4.43502


In [50]:
pitch_era_dtreg = round(mse(predicted["actual"], predicted["predicted"]), 5)
pitching = pitching.join(predicted, how='outer').fillna(0)
pitching = pitching.drop('actual', axis=1)
pitching.rename(columns = {'predicted':'ERA_DTR'}, inplace = True)

#### Strikeouts

In [51]:
#####
# - FUNCTION TO SPLIT BETWEEN TRAINING AND TEST SETS AND TO RUN THROUGH THE MODEL
#####
def pitch_so_model(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    years = sorted(data["Year"].unique())
    
    for i in range(start, len(years), step):
        current_year = years[i]
        train = data[data["Year"] < current_year]
        test = data[data["Year"] == current_year]
                
        model.fit(train[predictors], train["Nxt_SO"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["Nxt_SO"], preds], axis=1)
        combined.columns = ["actual", "predicted"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [52]:
predicted = pitch_so_model(pitching, dtr, pitch_so_preds)
predicted2 = pitch_so_model(pitching, dtr2, pitch_so_preds)
predicted3 = pitch_so_model(pitching, dtr3, pitch_so_preds)
predicted4 = pitch_so_model(pitching, dtr4, pitch_so_preds)
predicted5 = pitch_so_model(pitching, dtr5, pitch_so_preds)

predicted6 = pitch_so_model(pitching, dtr6, pitch_so_preds)
predicted7 = pitch_so_model(pitching, dtr7, pitch_so_preds)
predicted8 = pitch_so_model(pitching, dtr8, pitch_so_preds)
predicted9 = pitch_so_model(pitching, dtr9, pitch_so_preds)
predicted10 = pitch_so_model(pitching, dtr10, pitch_so_preds)

In [53]:
print('Decision Tree SO 01 MSE:',round(mse(predicted["actual"], predicted["predicted"]), 5))
print('Decision Tree SO 02 MSE:',round(mse(predicted2["actual"], predicted2["predicted"]), 5))
print('Decision Tree SO 03 MSE:',round(mse(predicted3["actual"], predicted3["predicted"]), 5))
print('Decision Tree SO 04 MSE:',round(mse(predicted4["actual"], predicted4["predicted"]), 5))
print('Decision Tree SO 05 MSE:',round(mse(predicted5["actual"], predicted5["predicted"]), 5))
print('Decision Tree SO 06 MSE:',round(mse(predicted6["actual"], predicted6["predicted"]), 5))
print('Decision Tree SO 07 MSE:',round(mse(predicted7["actual"], predicted7["predicted"]), 5))
print('Decision Tree SO 08 MSE:',round(mse(predicted8["actual"], predicted8["predicted"]), 5))
print('Decision Tree SO 09 MSE:',round(mse(predicted9["actual"], predicted9["predicted"]), 5))
print('Decision Tree SO 10 MSE:',round(mse(predicted10["actual"], predicted10["predicted"]), 5))

Decision Tree SO 01 MSE: 1521.98948
Decision Tree SO 02 MSE: 1426.53291
Decision Tree SO 03 MSE: 1402.50113
Decision Tree SO 04 MSE: 1432.64033
Decision Tree SO 05 MSE: 1462.02276
Decision Tree SO 06 MSE: 1517.95807
Decision Tree SO 07 MSE: 1573.58905
Decision Tree SO 08 MSE: 1660.77999
Decision Tree SO 09 MSE: 1715.81754
Decision Tree SO 10 MSE: 1797.68405


In [54]:
pitch_so_dtreg = round(mse(predicted3["actual"], predicted3["predicted"]), 5)
pitching = pitching.join(predicted3, how='outer').fillna(0)
pitching = pitching.drop('actual', axis=1)
pitching.rename(columns = {'predicted':'SO_DTR'}, inplace = True)

#### WHIP

In [55]:
#####
# - FUNCTION TO SPLIT BETWEEN TRAINING AND TEST SETS AND TO RUN THROUGH THE MODEL
#####
def pitch_whip_model(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    years = sorted(data["Year"].unique())
    
    for i in range(start, len(years), step):
        current_year = years[i]
        train = data[data["Year"] < current_year]
        test = data[data["Year"] == current_year]
                
        model.fit(train[predictors], train["Nxt_WHIP"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["Nxt_WHIP"], preds], axis=1)
        combined.columns = ["actual", "predicted"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [56]:
predicted = pitch_whip_model(pitching, dtr, pitch_whip_preds)
predicted2 = pitch_whip_model(pitching, dtr2, pitch_whip_preds)
predicted3 = pitch_whip_model(pitching, dtr3, pitch_whip_preds)
predicted4 = pitch_whip_model(pitching, dtr4, pitch_whip_preds)
predicted5 = pitch_whip_model(pitching, dtr5, pitch_whip_preds)

predicted6 = pitch_whip_model(pitching, dtr6, pitch_whip_preds)
predicted7 = pitch_whip_model(pitching, dtr7, pitch_whip_preds)
predicted8 = pitch_whip_model(pitching, dtr8, pitch_whip_preds)
predicted9 = pitch_whip_model(pitching, dtr9, pitch_whip_preds)
predicted10 = pitch_whip_model(pitching, dtr10, pitch_whip_preds)

In [57]:
print('Decision Tree WHIP 01 MSE:',round(mse(predicted["actual"], predicted["predicted"]), 5))
print('Decision Tree WHIP 02 MSE:',round(mse(predicted2["actual"], predicted2["predicted"]), 5))
print('Decision Tree WHIP 03 MSE:',round(mse(predicted3["actual"], predicted3["predicted"]), 5))
print('Decision Tree WHIP 04 MSE:',round(mse(predicted4["actual"], predicted4["predicted"]), 5))
print('Decision Tree WHIP 05 MSE:',round(mse(predicted5["actual"], predicted5["predicted"]), 5))
print('Decision Tree WHIP 06 MSE:',round(mse(predicted6["actual"], predicted6["predicted"]), 5))
print('Decision Tree WHIP 07 MSE:',round(mse(predicted7["actual"], predicted7["predicted"]), 5))
print('Decision Tree WHIP 08 MSE:',round(mse(predicted8["actual"], predicted8["predicted"]), 5))
print('Decision Tree WHIP 09 MSE:',round(mse(predicted9["actual"], predicted9["predicted"]), 5))
print('Decision Tree WHIP 10 MSE:',round(mse(predicted10["actual"], predicted10["predicted"]), 5))

Decision Tree WHIP 01 MSE: 0.10122
Decision Tree WHIP 02 MSE: 0.10075
Decision Tree WHIP 03 MSE: 0.10296
Decision Tree WHIP 04 MSE: 0.10461
Decision Tree WHIP 05 MSE: 0.1078
Decision Tree WHIP 06 MSE: 0.11072
Decision Tree WHIP 07 MSE: 0.11618
Decision Tree WHIP 08 MSE: 0.11975
Decision Tree WHIP 09 MSE: 0.12361
Decision Tree WHIP 10 MSE: 0.12827


In [58]:
pitch_whip_dtreg = round(mse(predicted2["actual"], predicted2["predicted"]), 5)
pitching = pitching.join(predicted2, how='outer').fillna(0)
pitching = pitching.drop('actual', axis=1)
pitching.rename(columns = {'predicted':'WHIP_DTR'}, inplace = True)

#### Walks

In [59]:
#####
# - FUNCTION TO SPLIT BETWEEN TRAINING AND TEST SETS AND TO RUN THROUGH THE MODEL
#####
def pitch_bb_model(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    years = sorted(data["Year"].unique())
    
    for i in range(start, len(years), step):
        current_year = years[i]
        train = data[data["Year"] < current_year]
        test = data[data["Year"] == current_year]
                
        model.fit(train[predictors], train["Nxt_BB"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["Nxt_BB"], preds], axis=1)
        combined.columns = ["actual", "predicted"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [60]:
predicted = pitch_bb_model(pitching, dtr, pitch_bb_preds)
predicted2 = pitch_bb_model(pitching, dtr2, pitch_bb_preds)
predicted3 = pitch_bb_model(pitching, dtr3, pitch_bb_preds)
predicted4 = pitch_bb_model(pitching, dtr4, pitch_bb_preds)
predicted5 = pitch_bb_model(pitching, dtr5, pitch_bb_preds)

predicted6 = pitch_bb_model(pitching, dtr6, pitch_bb_preds)
predicted7 = pitch_bb_model(pitching, dtr7, pitch_bb_preds)
predicted8 = pitch_bb_model(pitching, dtr8, pitch_bb_preds)
predicted9 = pitch_bb_model(pitching, dtr9, pitch_bb_preds)
predicted10 = pitch_bb_model(pitching, dtr10, pitch_bb_preds)

In [61]:
print('Decision Tree BB 01 MSE:',round(mse(predicted["actual"], predicted["predicted"]), 5))
print('Decision Tree BB 02 MSE:',round(mse(predicted2["actual"], predicted2["predicted"]), 5))
print('Decision Tree BB 03 MSE:',round(mse(predicted3["actual"], predicted3["predicted"]), 5))
print('Decision Tree BB 04 MSE:',round(mse(predicted4["actual"], predicted4["predicted"]), 5))
print('Decision Tree BB 05 MSE:',round(mse(predicted5["actual"], predicted5["predicted"]), 5))
print('Decision Tree BB 06 MSE:',round(mse(predicted6["actual"], predicted6["predicted"]), 5))
print('Decision Tree BB 07 MSE:',round(mse(predicted7["actual"], predicted7["predicted"]), 5))
print('Decision Tree BB 08 MSE:',round(mse(predicted8["actual"], predicted8["predicted"]), 5))
print('Decision Tree BB 09 MSE:',round(mse(predicted9["actual"], predicted9["predicted"]), 5))
print('Decision Tree BB 10 MSE:',round(mse(predicted10["actual"], predicted10["predicted"]), 5))

Decision Tree BB 01 MSE: 344.07915
Decision Tree BB 02 MSE: 325.47347
Decision Tree BB 03 MSE: 317.02254
Decision Tree BB 04 MSE: 316.87505
Decision Tree BB 05 MSE: 326.42225
Decision Tree BB 06 MSE: 342.48016
Decision Tree BB 07 MSE: 355.83995
Decision Tree BB 08 MSE: 370.20364
Decision Tree BB 09 MSE: 390.65571
Decision Tree BB 10 MSE: 409.85177


In [62]:
pitch_bb_dtreg = round(mse(predicted4["actual"], predicted4["predicted"]), 5)
pitching = pitching.join(predicted4, how='outer').fillna(0)
pitching = pitching.drop('actual', axis=1)
pitching.rename(columns = {'predicted':'BB_DTR'}, inplace = True)

#### Wins

In [63]:
#####
# - FUNCTION TO SPLIT BETWEEN TRAINING AND TEST SETS AND TO RUN THROUGH THE MODEL
#####
def pitch_w_model(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    years = sorted(data["Year"].unique())
    
    for i in range(start, len(years), step):
        current_year = years[i]
        train = data[data["Year"] < current_year]
        test = data[data["Year"] == current_year]
                
        model.fit(train[predictors], train["Nxt_W"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["Nxt_W"], preds], axis=1)
        combined.columns = ["actual", "predicted"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [64]:
predicted = pitch_w_model(pitching, dtr, pitch_w_preds)
predicted2 = pitch_w_model(pitching, dtr2, pitch_w_preds)
predicted3 = pitch_w_model(pitching, dtr3, pitch_w_preds)
predicted4 = pitch_w_model(pitching, dtr4, pitch_w_preds)
predicted5 = pitch_w_model(pitching, dtr5, pitch_w_preds)

predicted6 = pitch_w_model(pitching, dtr6, pitch_w_preds)
predicted7 = pitch_w_model(pitching, dtr7, pitch_w_preds)
predicted8 = pitch_w_model(pitching, dtr8, pitch_w_preds)
predicted9 = pitch_w_model(pitching, dtr9, pitch_w_preds)
predicted10 = pitch_w_model(pitching, dtr10, pitch_w_preds)

In [65]:
print('Decision Tree W 01 MSE:',round(mse(predicted["actual"], predicted["predicted"]), 5))
print('Decision Tree W 02 MSE:',round(mse(predicted2["actual"], predicted2["predicted"]), 5))
print('Decision Tree W 03 MSE:',round(mse(predicted3["actual"], predicted3["predicted"]), 5))
print('Decision Tree W 04 MSE:',round(mse(predicted4["actual"], predicted4["predicted"]), 5))
print('Decision Tree W 05 MSE:',round(mse(predicted5["actual"], predicted5["predicted"]), 5))
print('Decision Tree W 06 MSE:',round(mse(predicted6["actual"], predicted6["predicted"]), 5))
print('Decision Tree W 07 MSE:',round(mse(predicted7["actual"], predicted7["predicted"]), 5))
print('Decision Tree W 08 MSE:',round(mse(predicted8["actual"], predicted8["predicted"]), 5))
print('Decision Tree W 09 MSE:',round(mse(predicted9["actual"], predicted9["predicted"]), 5))
print('Decision Tree W 10 MSE:',round(mse(predicted10["actual"], predicted10["predicted"]), 5))

Decision Tree W 01 MSE: 15.23411
Decision Tree W 02 MSE: 14.48802
Decision Tree W 03 MSE: 14.37522
Decision Tree W 04 MSE: 14.3883
Decision Tree W 05 MSE: 14.7918
Decision Tree W 06 MSE: 15.35012
Decision Tree W 07 MSE: 16.05272
Decision Tree W 08 MSE: 16.73595
Decision Tree W 09 MSE: 17.57783
Decision Tree W 10 MSE: 18.2834


In [66]:
pitch_w_dtreg = round(mse(predicted3["actual"], predicted3["predicted"]), 5)
pitching = pitching.join(predicted3, how='outer').fillna(0)
pitching = pitching.drop('actual', axis=1)
pitching.rename(columns = {'predicted':'W_DTR'}, inplace = True)

#### Saves

In [67]:
#####
# - FUNCTION TO SPLIT BETWEEN TRAINING AND TEST SETS AND TO RUN THROUGH THE MODEL
#####
def pitch_sv_model(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    years = sorted(data["Year"].unique())
    
    for i in range(start, len(years), step):
        current_year = years[i]
        train = data[data["Year"] < current_year]
        test = data[data["Year"] == current_year]
                
        model.fit(train[predictors], train["Nxt_SV"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["Nxt_SV"], preds], axis=1)
        combined.columns = ["actual", "predicted"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [68]:
predicted = pitch_sv_model(pitching, dtr, pitch_sv_preds)
predicted2 = pitch_sv_model(pitching, dtr2, pitch_sv_preds)
predicted3 = pitch_sv_model(pitching, dtr3, pitch_sv_preds)
predicted4 = pitch_sv_model(pitching, dtr4, pitch_sv_preds)
predicted5 = pitch_sv_model(pitching, dtr5, pitch_sv_preds)

predicted6 = pitch_sv_model(pitching, dtr6, pitch_sv_preds)
predicted7 = pitch_sv_model(pitching, dtr7, pitch_sv_preds)
predicted8 = pitch_sv_model(pitching, dtr8, pitch_sv_preds)
predicted9 = pitch_sv_model(pitching, dtr9, pitch_sv_preds)
predicted10 = pitch_sv_model(pitching, dtr10, pitch_sv_preds)

In [69]:
print('Decision Tree SV 01 MSE:',round(mse(predicted["actual"], predicted["predicted"]), 5))
print('Decision Tree SV 02 MSE:',round(mse(predicted2["actual"], predicted2["predicted"]), 5))
print('Decision Tree SV 03 MSE:',round(mse(predicted3["actual"], predicted3["predicted"]), 5))
print('Decision Tree SV 04 MSE:',round(mse(predicted4["actual"], predicted4["predicted"]), 5))
print('Decision Tree SV 05 MSE:',round(mse(predicted5["actual"], predicted5["predicted"]), 5))
print('Decision Tree SV 06 MSE:',round(mse(predicted6["actual"], predicted6["predicted"]), 5))
print('Decision Tree SV 07 MSE:',round(mse(predicted7["actual"], predicted7["predicted"]), 5))
print('Decision Tree SV 08 MSE:',round(mse(predicted8["actual"], predicted8["predicted"]), 5))
print('Decision Tree SV 09 MSE:',round(mse(predicted9["actual"], predicted9["predicted"]), 5))
print('Decision Tree SV 10 MSE:',round(mse(predicted10["actual"], predicted10["predicted"]), 5))

Decision Tree SV 01 MSE: 30.47403
Decision Tree SV 02 MSE: 29.99157
Decision Tree SV 03 MSE: 30.68917
Decision Tree SV 04 MSE: 32.30139
Decision Tree SV 05 MSE: 33.40941
Decision Tree SV 06 MSE: 35.38465
Decision Tree SV 07 MSE: 36.82148
Decision Tree SV 08 MSE: 37.62988
Decision Tree SV 09 MSE: 39.1944
Decision Tree SV 10 MSE: 40.44137


In [70]:
pitch_sv_dtreg = round(mse(predicted2["actual"], predicted2["predicted"]), 5)
pitching = pitching.join(predicted2, how='outer').fillna(0)
pitching = pitching.drop('actual', axis=1)
pitching.rename(columns = {'predicted':'SV_DTR'}, inplace = True)

In [71]:
pitching

,Name,Age,Tm,W,L,W-L%,ERA,G,GS,GF,CG,SHO,SV,IP,H,R,ER,HR,BB,IBB,SO,HBP,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Year,Nxt_ERA,Nxt_SO,Nxt_WHIP,Nxt_BB,Nxt_W,Nxt_SV,ERA_DTR,SO_DTR,WHIP_DTR,BB_DTR,W_DTR,SV_DTR
0,A.J. Achter,0.233333,MIN,0.037037,0.000000,1.000,0.110236,0.019802,0.000000,0.011905,0.000000,0.0,0.0,0.026652,0.034211,0.036364,0.020408,0.04,0.014706,0.000000,0.013055,0.000000,0.0000,0.000000,0.026161,0.077377,0.287988,0.144087,0.303125,0.137931,0.046296,0.220430,0.0668,2014,6.75,14.0,1.350,6.0,0.0,0.0,5.113952,32.940217,1.476450,18.584777,2.469477,0.549599
1,A.J. Achter,0.266667,MIN,0.000000,0.045455,0.000,0.240720,0.059406,0.000000,0.047619,0.000000,0.0,0.0,0.032249,0.028947,0.054545,0.061224,0.08,0.029412,0.041667,0.036554,0.000000,0.0000,0.000000,0.032047,0.032424,0.330439,0.121699,0.196875,0.232759,0.075926,0.510753,0.0932,2015,3.11,14.0,1.460,12.0,1.0,0.0,5.115102,33.071984,1.457083,17.685851,2.452248,0.542181
3,A.J. Burnett,0.133333,FLA,0.148148,0.090909,0.667,0.118110,0.019802,0.142857,0.000000,0.000000,0.0,0.0,0.106876,0.094737,0.133333,0.102041,0.06,0.122549,0.083333,0.086162,0.000000,0.0000,0.000000,0.113146,0.078113,0.234925,0.138921,0.196875,0.060345,0.100000,0.387097,0.0528,1999,4.79,57.0,1.500,44.0,3.0,0.0,4.543846,61.087603,1.539702,36.365269,5.146341,0.709539
4,A.J. Burnett,0.166667,FLA,0.111111,0.318182,0.300,0.167229,0.079208,0.265306,0.000000,0.000000,0.0,0.0,0.216418,0.207895,0.272727,0.292517,0.16,0.215686,0.125000,0.148825,0.095238,0.0000,0.076923,0.232178,0.054532,0.253256,0.138921,0.215625,0.077586,0.088889,0.333333,0.0520,2000,4.05,128.0,1.315,83.0,11.0,0.0,4.583168,60.349922,1.502068,36.294521,6.268229,0.516997
5,A.J. Burnett,0.200000,FLA,0.407407,0.545455,0.478,0.139483,0.217822,0.551020,0.000000,0.066667,0.1,0.0,0.458689,0.378947,0.490909,0.523810,0.40,0.406863,0.125000,0.334204,0.333333,0.0625,0.269231,0.473512,0.064112,0.250844,0.117681,0.178125,0.086207,0.079630,0.354839,0.0616,2001,3.30,203.0,1.189,90.0,12.0,0.0,4.594213,109.832080,1.471056,71.987705,9.460055,0.685406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23609,Zane Smith*,0.433333,PIT,0.296296,0.363636,0.500,0.102362,0.178218,0.448980,0.000000,0.133333,0.3,0.0,0.373134,0.360526,0.333333,0.319728,0.16,0.093137,0.125000,0.146214,0.095238,0.0000,0.000000,0.364290,0.070744,0.180415,0.094489,0.218750,0.043103,0.022222,0.193548,0.1180,1992,4.55,32.0,1.434,22.0,3.0,0.0,3.493447,59.379242,1.226647,50.644068,9.341040,0.590836
23610,Zane Smith*,0.466667,PIT,0.111111,0.318182,0.300,0.158230,0.089109,0.285714,0.000000,0.033333,0.0,0.0,0.218550,0.252632,0.254545,0.278912,0.10,0.107843,0.125000,0.083551,0.000000,0.0000,0.076923,0.224984,0.051584,0.210806,0.131343,0.271875,0.043103,0.044444,0.188172,0.0580,1993,3.27,57.0,1.248,34.0,10.0,0.0,4.141861,59.680529,1.419493,32.170418,5.381404,0.575998
23611,Zane Smith*,0.500000,PIT,0.370370,0.363636,0.556,0.110236,0.198020,0.489796,0.000000,0.066667,0.1,0.0,0.415778,0.423684,0.400000,0.380952,0.36,0.166667,0.291667,0.148825,0.000000,0.0000,0.076923,0.415958,0.084009,0.246985,0.109989,0.234375,0.086207,0.035185,0.177419,0.0672,1994,5.61,47.0,1.509,23.0,8.0,0.0,4.588625,45.753396,1.423863,36.232779,9.153595,0.555363
23612,Zane Smith*,0.533333,BOS,0.296296,0.363636,0.500,0.197975,0.188119,0.428571,0.000000,0.000000,0.0,0.0,0.291045,0.376316,0.466667,0.462585,0.14,0.112745,0.041667,0.122715,0.047619,0.0625,0.000000,0.310661,0.050847,0.207429,0.139954,0.309375,0.051724,0.035185,0.204301,0.0816,1995,5.08,47.0,1.500,21.0,4.0,0.0,4.234392,46.241810,1.323820,24.491945,6.355856,0.541804


In [72]:
pitching[['Nxt_ERA','Nxt_SO','Nxt_WHIP','Nxt_BB','Nxt_W','Nxt_SV']].describe()

,Nxt_ERA,Nxt_SO,Nxt_WHIP,Nxt_BB,Nxt_W,Nxt_SV
count,19733.00000,19733.000000,19733.000000,19733.000000,19733.000000,19733.000000
mean,4.45989,65.571479,1.423791,32.634166,5.196169,2.631176
std,1.94710,51.579764,0.327287,23.267034,4.915708,7.328233
min,0.33000,0.000000,0.273000,0.000000,0.000000,0.000000
25%,3.29000,27.000000,1.230000,15.000000,1.000000,0.000000
50%,4.13000,53.000000,1.378000,27.000000,4.000000,0.000000
75%,5.19000,90.000000,1.557000,46.000000,8.000000,1.000000
max,37.13000,383.000000,9.000000,204.000000,27.000000,62.000000


In [73]:
#####
# - UNSCALING DATA (INVERSE TRANSFORM CAUSED ERRORS)
#####

## - ERA

pitching['ERA_DTR'] = round(pitching['ERA_DTR'], 2)

## - SO
pitch_so_dtr = pitching[['SO_DTR']]
#scaled from 0 to 383
min = 0
max = 383
pitch_so_dtr * min + (max - min)

pitching['SO_DTR'] = pitch_so_dtr.astype(int)

## - WHIP

pitching['WHIP_DTR'] = round(pitching['WHIP_DTR'], 2)


## - BB
pitch_bb_dtr = pitching[['BB_DTR']]
#scaled from 0 to 204
min = 0
max = 204
pitch_bb_dtr * min + (max - min)

pitching['BB_DTR'] = pitch_bb_dtr.astype(int)

## - W
pitch_w_dtr = pitching[['W_DTR']]
#scaled from 0 to 27
min = 0
max = 27
pitch_w_dtr * min + (max - min)

pitching['W_DTR'] = pitch_w_dtr.astype(int)

## - SV
pitch_sv_dtr = pitching[['SV_DTR']]
#scaled from 0 to 62
min = 0
max = 62
pitch_sv_dtr * min + (max - min)

pitching['SV_DTR'] = pitch_sv_dtr.astype(int)


pitching.head()

,Name,Age,Tm,W,L,W-L%,ERA,G,GS,GF,CG,SHO,SV,IP,H,R,ER,HR,BB,IBB,SO,HBP,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Year,Nxt_ERA,Nxt_SO,Nxt_WHIP,Nxt_BB,Nxt_W,Nxt_SV,ERA_DTR,SO_DTR,WHIP_DTR,BB_DTR,W_DTR,SV_DTR
0,A.J. Achter,0.233333,MIN,0.037037,0.000000,1.000,0.110236,0.019802,0.000000,0.011905,0.000000,0.0,0.0,0.026652,0.034211,0.036364,0.020408,0.04,0.014706,0.000000,0.013055,0.000000,0.0000,0.000000,0.026161,0.077377,0.287988,0.144087,0.303125,0.137931,0.046296,0.220430,0.0668,2014,6.75,14.0,1.350,6.0,0.0,0.0,5.11,32,1.48,18,2,0
1,A.J. Achter,0.266667,MIN,0.000000,0.045455,0.000,0.240720,0.059406,0.000000,0.047619,0.000000,0.0,0.0,0.032249,0.028947,0.054545,0.061224,0.08,0.029412,0.041667,0.036554,0.000000,0.0000,0.000000,0.032047,0.032424,0.330439,0.121699,0.196875,0.232759,0.075926,0.510753,0.0932,2015,3.11,14.0,1.460,12.0,1.0,0.0,5.12,33,1.46,17,2,0
3,A.J. Burnett,0.133333,FLA,0.148148,0.090909,0.667,0.118110,0.019802,0.142857,0.000000,0.000000,0.0,0.0,0.106876,0.094737,0.133333,0.102041,0.06,0.122549,0.083333,0.086162,0.000000,0.0000,0.000000,0.113146,0.078113,0.234925,0.138921,0.196875,0.060345,0.100000,0.387097,0.0528,1999,4.79,57.0,1.500,44.0,3.0,0.0,4.54,61,1.54,36,5,0
4,A.J. Burnett,0.166667,FLA,0.111111,0.318182,0.300,0.167229,0.079208,0.265306,0.000000,0.000000,0.0,0.0,0.216418,0.207895,0.272727,0.292517,0.16,0.215686,0.125000,0.148825,0.095238,0.0000,0.076923,0.232178,0.054532,0.253256,0.138921,0.215625,0.077586,0.088889,0.333333,0.0520,2000,4.05,128.0,1.315,83.0,11.0,0.0,4.58,60,1.50,36,6,0
5,A.J. Burnett,0.200000,FLA,0.407407,0.545455,0.478,0.139483,0.217822,0.551020,0.000000,0.066667,0.1,0.0,0.458689,0.378947,0.490909,0.523810,0.40,0.406863,0.125000,0.334204,0.333333,0.0625,0.269231,0.473512,0.064112,0.250844,0.117681,0.178125,0.086207,0.079630,0.354839,0.0616,2001,3.30,203.0,1.189,90.0,12.0,0.0,4.59,109,1.47,71,9,0


# Linear Regression Model

In [74]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression(n_jobs = -1)

## Batters

#### Batting Average

In [75]:
#####
# - FUNCTION TO SPLIT BETWEEN TRAINING AND TEST SETS AND TO RUN THROUGH THE MODEL
#####
from sklearn.compose import TransformedTargetRegressor


def bat_ba_model(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    years = sorted(data["Year"].unique())
    
    for i in range(start, len(years), step):
        current_year = years[i]
        train = data[data["Year"] < current_year]
        test = data[data["Year"] == current_year]
                
        model.fit(train[predictors], train["Nxt_BA"])
        
        preds = model.predict(test[predictors])

        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["Nxt_BA"], preds], axis=1)
        combined.columns = ["actual", "predicted"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [76]:
predicted = bat_ba_model(batting, lr, bat_ba_preds)

In [77]:
print('Linear Regression BA MSE:',round(mse(predicted["actual"], predicted["predicted"]), 5))

Linear Regression BA MSE: 0.00386


In [78]:
bat_ba_lreg = round(mse(predicted["actual"], predicted["predicted"]), 5)
batting = batting.join(predicted, how='outer').fillna(0)
batting = batting.drop('actual', axis=1)
batting.rename(columns = {'predicted':'BA_LR'}, inplace = True)

#### RBI

In [79]:
#####
# - FUNCTION TO SPLIT BETWEEN TRAINING AND TEST SETS AND TO RUN THROUGH THE MODEL
#####
def bat_rbi_model(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    years = sorted(data["Year"].unique())
    
    for i in range(start, len(years), step):
        current_year = years[i]
        train = data[data["Year"] < current_year]
        test = data[data["Year"] == current_year]
                
        model.fit(train[predictors], train["Nxt_RBI"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["Nxt_RBI"], preds], axis=1)
        combined.columns = ["actual", "predicted"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [80]:
predicted = bat_rbi_model(batting, lr, bat_rbi_preds)

In [81]:
print('Linear Regression RBI MSE:',round(mse(predicted["actual"], predicted["predicted"]), 5))

Linear Regression RBI MSE: 394.91091


In [82]:
bat_rbi_lreg = round(mse(predicted["actual"], predicted["predicted"]), 5)
batting = batting.join(predicted, how='outer').fillna(0)
batting = batting.drop('actual', axis=1)
batting.rename(columns = {'predicted':'RBI_LR'}, inplace = True)

#### Home Runs

In [83]:
#####
# - FUNCTION TO SPLIT BETWEEN TRAINING AND TEST SETS AND TO RUN THROUGH THE MODEL
#####
def bat_hr_model(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    years = sorted(data["Year"].unique())
    
    for i in range(start, len(years), step):
        current_year = years[i]
        train = data[data["Year"] < current_year]
        test = data[data["Year"] == current_year]
                
        model.fit(train[predictors], train["Nxt_HR"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["Nxt_HR"], preds], axis=1)
        combined.columns = ["actual", "predicted"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [84]:
predicted = bat_hr_model(batting, lr, bat_hr_preds)

In [85]:
print('Linear Regression HR MSE:',round(mse(predicted["actual"], predicted["predicted"]), 5))

Linear Regression HR MSE: 38.55819


In [86]:
bat_hr_lreg = round(mse(predicted["actual"], predicted["predicted"]), 5)
batting = batting.join(predicted, how='outer').fillna(0)
batting = batting.drop('actual', axis=1)
batting.rename(columns = {'predicted':'HR_LR'}, inplace = True)

#### Walks

In [87]:
#####
# - FUNCTION TO SPLIT BETWEEN TRAINING AND TEST SETS AND TO RUN THROUGH THE MODEL
#####
def bat_bb_model(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    years = sorted(data["Year"].unique())
    
    for i in range(start, len(years), step):
        current_year = years[i]
        train = data[data["Year"] < current_year]
        test = data[data["Year"] == current_year]
                
        model.fit(train[predictors], train["Nxt_BB"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["Nxt_BB"], preds], axis=1)
        combined.columns = ["actual", "predicted"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [88]:
predicted = bat_bb_model(batting, lr, bat_bb_preds)

In [89]:
print('Linear Regression BB MSE:',round(mse(predicted["actual"], predicted["predicted"]), 5))

Linear Regression BB MSE: 230.07081


In [90]:
bat_bb_lreg = round(mse(predicted["actual"], predicted["predicted"]), 5)
batting = batting.join(predicted, how='outer').fillna(0)
batting = batting.drop('actual', axis=1)
batting.rename(columns = {'predicted':'BB_LR'}, inplace = True)

#### Strikeouts

In [91]:
#####
# - FUNCTION TO SPLIT BETWEEN TRAINING AND TEST SETS AND TO RUN THROUGH THE MODEL
#####
def bat_so_model(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    years = sorted(data["Year"].unique())
    
    for i in range(start, len(years), step):
        current_year = years[i]
        train = data[data["Year"] < current_year]
        test = data[data["Year"] == current_year]
                
        model.fit(train[predictors], train["Nxt_SO"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["Nxt_SO"], preds], axis=1)
        combined.columns = ["actual", "predicted"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [92]:
predicted = bat_so_model(batting, lr, bat_so_preds)

In [93]:
print('Linear Regression SO MSE:',round(mse(predicted["actual"], predicted["predicted"]), 5))

Linear Regression SO MSE: 647.49604


In [94]:
bat_so_lreg = round(mse(predicted["actual"], predicted["predicted"]), 5)
batting = batting.join(predicted, how='outer').fillna(0)
batting = batting.drop('actual', axis=1)
batting.rename(columns = {'predicted':'SO_LR'}, inplace = True)

In [95]:
#####
# - UNSCALING DATA (INVERSE TRANSFORM CAUSED ERRORS)
#####

## - BA
batting[['BA_LR']] = round(batting[['BA_LR']], 3)

## - RBI
bat_rbi_lr = batting[['RBI_LR']]
#scaled from 0 to 165
min = 0
max = 165
bat_rbi_lr * min + (max - min)

batting['RBI_LR'] = bat_rbi_lr.astype(int)

## - HR
bat_hr_lr = batting[['HR_LR']]
#scaled from 0 to 73
min = 0
max = 73
bat_hr_lr * min + (max - min)

batting['HR_LR'] = bat_hr_lr.astype(int)

## - BB
bat_bb_lr = batting[['BB_LR']]
#scaled from 0 to 232
min = 0
max = 232
bat_bb_lr * min + (max - min)

batting['BB_LR'] = bat_bb_lr.astype(int)

## - SO
bat_so_lr = batting[['SO_LR']]
#scaled from 0 to 223
min = 0
max = 223
bat_so_lr * min + (max - min)

batting['SO_LR'] = bat_so_lr.astype(int)

batting['Nxt_RBI'] = batting['Nxt_RBI'].astype(int)
batting['Nxt_HR'] = batting['Nxt_HR'].astype(int)
batting['Nxt_BB'] = batting['Nxt_BB'].astype(int)
batting['Nxt_SO'] = batting['Nxt_SO'].astype(int)

In [96]:
batting.insert(len(batting.columns)-1, 'Nxt_BA', batting.pop('Nxt_BA'))
batting.insert(len(batting.columns)-1, 'BA_DTR', batting.pop('BA_DTR'))
batting.insert(len(batting.columns)-1, 'BA_LR', batting.pop('BA_LR'))

batting.insert(len(batting.columns)-1, 'Nxt_RBI', batting.pop('Nxt_RBI'))
batting.insert(len(batting.columns)-1, 'RBI_DTR', batting.pop('RBI_DTR'))
batting.insert(len(batting.columns)-1, 'RBI_LR', batting.pop('RBI_LR'))

batting.insert(len(batting.columns)-1, 'Nxt_HR', batting.pop('Nxt_HR'))
batting.insert(len(batting.columns)-1, 'HR_DTR', batting.pop('HR_DTR'))
batting.insert(len(batting.columns)-1, 'HR_LR', batting.pop('HR_LR'))

batting.insert(len(batting.columns)-1, 'Nxt_BB', batting.pop('Nxt_BB'))
batting.insert(len(batting.columns)-1, 'BB_DTR', batting.pop('BB_DTR'))
batting.insert(len(batting.columns)-1, 'BB_LR', batting.pop('BB_LR'))

batting.insert(len(batting.columns)-1, 'Nxt_SO', batting.pop('Nxt_SO'))
batting.insert(len(batting.columns)-1, 'SO_DTR', batting.pop('SO_DTR'))
batting.insert(len(batting.columns)-1, 'SO_LR', batting.pop('SO_LR'))

In [97]:
batting.head()

,Name,Age,Tm,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Year,Nxt_BA,BA_DTR,BA_LR,Nxt_RBI,RBI_DTR,RBI_LR,Nxt_HR,HR_DTR,HR_LR,Nxt_BB,BB_DTR,BB_LR,Nxt_SO,SO_DTR,SO_LR
0,A.J. Burnett,0.206897,FLA,0.067114,0.073454,0.067227,0.000000,0.015267,0.016949,0.000000,0.000000,0.000000,0.0,0.0,0.004310,0.121076,0.080,0.115,0.0500,0.080615,0.091627,0.011765,0.000000,0.02,0.175,0.0,0.0,2001,0.105,0.135,0.150,3,2,5,1,0,1,5,2,4,28,18,27
1,A.J. Burnett,0.241379,FLA,0.093960,0.086340,0.077031,0.006579,0.022901,0.033898,0.000000,0.013699,0.018182,0.0,0.0,0.021552,0.125561,0.105,0.177,0.0965,0.138733,0.156398,0.025882,0.027778,0.00,0.175,0.0,0.0,2002,0.138,0.132,0.167,1,5,9,0,2,2,0,4,8,11,17,31
2,A.J. Burnett,0.310345,FLA,0.020134,0.045103,0.037815,0.026316,0.015267,0.000000,0.000000,0.000000,0.006061,0.0,0.0,0.000000,0.049327,0.138,0.138,0.0690,0.103487,0.116904,0.009412,0.027778,0.00,0.200,0.0,0.0,2004,0.147,0.142,0.155,2,5,3,1,0,0,1,3,2,34,17,15
3,A.J. Burnett,0.344828,FLA,0.107383,0.099227,0.092437,0.019737,0.038168,0.033898,0.086957,0.013699,0.012121,0.0,0.0,0.004310,0.152466,0.147,0.171,0.1395,0.169104,0.187994,0.044706,0.027778,0.02,0.225,0.0,0.0,2005,0.063,0.190,0.168,2,5,6,0,2,1,3,3,4,36,24,33
4,A.J. Burnett,0.586207,PIT,0.093960,0.088918,0.085434,0.006579,0.015267,0.000000,0.000000,0.000000,0.012121,0.0,0.0,0.012931,0.161435,0.063,0.106,0.0315,0.063742,0.077409,0.009412,0.027778,0.00,0.125,0.0,0.0,2012,0.068,0.138,0.138,2,3,-1,0,0,0,2,1,0,33,20,23


## Pitchers

#### ERA

In [98]:
#####
# - FUNCTION TO SPLIT BETWEEN TRAINING AND TEST SETS AND TO RUN THROUGH THE MODEL
#####
def pitch_era_model(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    years = sorted(data["Year"].unique())
    
    for i in range(start, len(years), step):
        current_year = years[i]
        train = data[data["Year"] < current_year]
        test = data[data["Year"] == current_year]
                
        model.fit(train[predictors], train["Nxt_ERA"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["Nxt_ERA"], preds], axis=1)
        combined.columns = ["actual", "predicted"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [99]:
predicted = pitch_era_model(pitching, lr, pitch_era_preds)

In [100]:
print('Linear Regression ERA MSE:',round(mse(predicted["actual"], predicted["predicted"]), 5))

Linear Regression ERA MSE: 3.55128


In [101]:
pitch_era_lreg = round(mse(predicted["actual"], predicted["predicted"]), 5)
pitching = pitching.join(predicted, how='outer').fillna(0)
pitching = pitching.drop('actual', axis=1)
pitching.rename(columns = {'predicted':'ERA_LR'}, inplace = True)

#### Strikeouts

In [102]:
#####
# - FUNCTION TO SPLIT BETWEEN TRAINING AND TEST SETS AND TO RUN THROUGH THE MODEL
#####
def pitch_so_model(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    years = sorted(data["Year"].unique())
    
    for i in range(start, len(years), step):
        current_year = years[i]
        train = data[data["Year"] < current_year]
        test = data[data["Year"] == current_year]
                
        model.fit(train[predictors], train["Nxt_SO"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["Nxt_SO"], preds], axis=1)
        combined.columns = ["actual", "predicted"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [103]:
predicted = pitch_so_model(pitching, lr, pitch_so_preds)

In [104]:
print('Linear Regression SO MSE:',round(mse(predicted["actual"], predicted["predicted"]), 5))

Linear Regression SO MSE: 1311.38268


In [105]:
pitch_so_lreg = round(mse(predicted["actual"], predicted["predicted"]), 5)
pitching = pitching.join(predicted, how='outer').fillna(0)
pitching = pitching.drop('actual', axis=1)
pitching.rename(columns = {'predicted':'SO_LR'}, inplace = True)

#### WHIP

In [106]:
#####
# - FUNCTION TO SPLIT BETWEEN TRAINING AND TEST SETS AND TO RUN THROUGH THE MODEL
#####
def pitch_whip_model(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    years = sorted(data["Year"].unique())
    
    for i in range(start, len(years), step):
        current_year = years[i]
        train = data[data["Year"] < current_year]
        test = data[data["Year"] == current_year]
                
        model.fit(train[predictors], train["Nxt_WHIP"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["Nxt_WHIP"], preds], axis=1)
        combined.columns = ["actual", "predicted"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [107]:
predicted = pitch_whip_model(pitching, lr, pitch_whip_preds)

In [108]:
print('Linear Regression WHIP MSE:',round(mse(predicted["actual"], predicted["predicted"]), 5))

Linear Regression WHIP MSE: 0.09735


In [109]:
pitch_whip_lreg = round(mse(predicted["actual"], predicted["predicted"]), 5)
pitching = pitching.join(predicted, how='outer').fillna(0)
pitching = pitching.drop('actual', axis=1)
pitching.rename(columns = {'predicted':'WHIP_LR'}, inplace = True)

#### Walks

In [110]:
#####
# - FUNCTION TO SPLIT BETWEEN TRAINING AND TEST SETS AND TO RUN THROUGH THE MODEL
#####
def pitch_bb_model(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    years = sorted(data["Year"].unique())
    
    for i in range(start, len(years), step):
        current_year = years[i]
        train = data[data["Year"] < current_year]
        test = data[data["Year"] == current_year]
                
        model.fit(train[predictors], train["Nxt_BB"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["Nxt_BB"], preds], axis=1)
        combined.columns = ["actual", "predicted"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [111]:
predicted = pitch_bb_model(pitching, lr, pitch_bb_preds)

In [112]:
print('Linear Regression BB MSE:',round(mse(predicted["actual"], predicted["predicted"]), 5))

Linear Regression BB MSE: 290.21958


In [113]:
pitch_bb_lreg = round(mse(predicted["actual"], predicted["predicted"]), 5)
pitching = pitching.join(predicted, how='outer').fillna(0)
pitching = pitching.drop('actual', axis=1)
pitching.rename(columns = {'predicted':'BB_LR'}, inplace = True)

#### Wins

In [114]:
#####
# - FUNCTION TO SPLIT BETWEEN TRAINING AND TEST SETS AND TO RUN THROUGH THE MODEL
#####
def pitch_w_model(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    years = sorted(data["Year"].unique())
    
    for i in range(start, len(years), step):
        current_year = years[i]
        train = data[data["Year"] < current_year]
        test = data[data["Year"] == current_year]
                
        model.fit(train[predictors], train["Nxt_W"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["Nxt_W"], preds], axis=1)
        combined.columns = ["actual", "predicted"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [115]:
predicted = pitch_w_model(pitching, lr, pitch_w_preds)

In [116]:
print('Linear Regression W MSE:',round(mse(predicted["actual"], predicted["predicted"]), 5))

Linear Regression W MSE: 13.42998


In [117]:
pitch_w_lreg = round(mse(predicted["actual"], predicted["predicted"]), 5)
pitching = pitching.join(predicted, how='outer').fillna(0)
pitching = pitching.drop('actual', axis=1)
pitching.rename(columns = {'predicted':'W_LR'}, inplace = True)

#### Saves

In [118]:
#####
# - FUNCTION TO SPLIT BETWEEN TRAINING AND TEST SETS AND TO RUN THROUGH THE MODEL
#####
def pitch_sv_model(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    years = sorted(data["Year"].unique())
    
    for i in range(start, len(years), step):
        current_year = years[i]
        train = data[data["Year"] < current_year]
        test = data[data["Year"] == current_year]
                
        model.fit(train[predictors], train["Nxt_SV"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["Nxt_SV"], preds], axis=1)
        combined.columns = ["actual", "predicted"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [119]:
predicted = pitch_sv_model(pitching, lr, pitch_sv_preds)

In [120]:
print('Linear Regression SV MSE:',round(mse(predicted["actual"], predicted["predicted"]), 5))

Linear Regression SV MSE: 28.30667


In [121]:
pitch_sv_lreg = round(mse(predicted["actual"], predicted["predicted"]), 5)
pitching = pitching.join(predicted, how='outer').fillna(0)
pitching = pitching.drop('actual', axis=1)
pitching.rename(columns = {'predicted':'SV_LR'}, inplace = True)

In [122]:
pitching

,Name,Age,Tm,W,L,W-L%,ERA,G,GS,GF,CG,SHO,SV,IP,H,R,ER,HR,BB,IBB,SO,HBP,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Year,Nxt_ERA,Nxt_SO,Nxt_WHIP,Nxt_BB,Nxt_W,Nxt_SV,ERA_DTR,SO_DTR,WHIP_DTR,BB_DTR,W_DTR,SV_DTR,ERA_LR,SO_LR,WHIP_LR,BB_LR,W_LR,SV_LR
0,A.J. Achter,0.233333,MIN,0.037037,0.000000,1.000,0.110236,0.019802,0.000000,0.011905,0.000000,0.0,0.0,0.026652,0.034211,0.036364,0.020408,0.04,0.014706,0.000000,0.013055,0.000000,0.0000,0.000000,0.026161,0.077377,0.287988,0.144087,0.303125,0.137931,0.046296,0.220430,0.0668,2014,6.75,14.0,1.350,6.0,0.0,0.0,5.11,32,1.48,18,2,0,4.970499,39.619037,1.493819,21.971424,3.080639,0.150127
1,A.J. Achter,0.266667,MIN,0.000000,0.045455,0.000,0.240720,0.059406,0.000000,0.047619,0.000000,0.0,0.0,0.032249,0.028947,0.054545,0.061224,0.08,0.029412,0.041667,0.036554,0.000000,0.0000,0.000000,0.032047,0.032424,0.330439,0.121699,0.196875,0.232759,0.075926,0.510753,0.0932,2015,3.11,14.0,1.460,12.0,1.0,0.0,5.12,33,1.46,17,2,0,4.754774,37.644682,1.464327,19.649664,2.442997,1.314805
3,A.J. Burnett,0.133333,FLA,0.148148,0.090909,0.667,0.118110,0.019802,0.142857,0.000000,0.000000,0.0,0.0,0.106876,0.094737,0.133333,0.102041,0.06,0.122549,0.083333,0.086162,0.000000,0.0000,0.000000,0.113146,0.078113,0.234925,0.138921,0.196875,0.060345,0.100000,0.387097,0.0528,1999,4.79,57.0,1.500,44.0,3.0,0.0,4.54,61,1.54,36,5,0,4.846718,63.303324,1.501743,39.561270,5.120677,1.917496
4,A.J. Burnett,0.166667,FLA,0.111111,0.318182,0.300,0.167229,0.079208,0.265306,0.000000,0.000000,0.0,0.0,0.216418,0.207895,0.272727,0.292517,0.16,0.215686,0.125000,0.148825,0.095238,0.0000,0.076923,0.232178,0.054532,0.253256,0.138921,0.215625,0.077586,0.088889,0.333333,0.0520,2000,4.05,128.0,1.315,83.0,11.0,0.0,4.58,60,1.50,36,6,0,5.000651,70.379341,1.496370,43.853405,5.468469,0.818671
5,A.J. Burnett,0.200000,FLA,0.407407,0.545455,0.478,0.139483,0.217822,0.551020,0.000000,0.066667,0.1,0.0,0.458689,0.378947,0.490909,0.523810,0.40,0.406863,0.125000,0.334204,0.333333,0.0625,0.269231,0.473512,0.064112,0.250844,0.117681,0.178125,0.086207,0.079630,0.354839,0.0616,2001,3.30,203.0,1.189,90.0,12.0,0.0,4.59,109,1.47,71,9,0,4.743655,118.550525,1.440667,68.568403,9.200324,0.379679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23609,Zane Smith*,0.433333,PIT,0.296296,0.363636,0.500,0.102362,0.178218,0.448980,0.000000,0.133333,0.3,0.0,0.373134,0.360526,0.333333,0.319728,0.16,0.093137,0.125000,0.146214,0.095238,0.0000,0.000000,0.364290,0.070744,0.180415,0.094489,0.218750,0.043103,0.022222,0.193548,0.1180,1992,4.55,32.0,1.434,22.0,3.0,0.0,3.49,59,1.23,50,9,0,3.968466,62.644289,1.317254,31.504239,7.944126,0.795350
23610,Zane Smith*,0.466667,PIT,0.111111,0.318182,0.300,0.158230,0.089109,0.285714,0.000000,0.033333,0.0,0.0,0.218550,0.252632,0.254545,0.278912,0.10,0.107843,0.125000,0.083551,0.000000,0.0000,0.076923,0.224984,0.051584,0.210806,0.131343,0.271875,0.043103,0.044444,0.188172,0.0580,1993,3.27,57.0,1.248,34.0,10.0,0.0,4.14,59,1.42,32,5,0,4.520782,41.723230,1.446818,25.174733,4.877704,0.255074
23611,Zane Smith*,0.500000,PIT,0.370370,0.363636,0.556,0.110236,0.198020,0.489796,0.000000,0.066667,0.1,0.0,0.415778,0.423684,0.400000,0.380952,0.36,0.166667,0.291667,0.148825,0.000000,0.0000,0.076923,0.415958,0.084009,0.246985,0.109989,0.234375,0.086207,0.035185,0.177419,0.0672,1994,5.61,47.0,1.509,23.0,8.0,0.0,4.59,45,1.42,36,9,0,4.674788,45.667477,1.378902,27.444026,7.092849,-0.516476
23612,Zane Smith*,0.533333,BOS,0.296296,0.363636,0.500,0.197975,0.188119,0.428571,0.000000,0.000000,0.0,0.0,0.291045,0.376316,0.466667,0.462585,0.14,0.112745,0.041667,0.122715,0.047619,0.0625,0.000000,0.310661,0.050847,0.207429,0.139954,0.309375,0.051724,0.035185,0.204301,0.0816,1995,5.08,47.0,1.500,21.0,4.0,0.0,4.23,46,1.32,24,6,0,4.632200,45.211382,1.417003,23.580488,5.616458,0.480199


In [123]:
#####
# - UNSCALING DATA (INVERSE TRANSFORM CAUSED ERRORS)
#####

## - ERA

pitching['ERA_LR'] = round(pitching['ERA_LR'], 2)

## - SO
pitch_so_lr = pitching[['SO_LR']]
#scaled from 0 to 383
min = 0
max = 383
pitch_so_lr * min + (max - min)

pitching['SO_LR'] = pitch_so_lr.astype(int)

## - WHIP

pitching['WHIP_LR'] = round(pitching['WHIP_LR'], 3)


## - BB
pitch_bb_lr = pitching[['BB_LR']]
#scaled from 0 to 204
min = 0
max = 204
pitch_bb_lr * min + (max - min)

pitching['BB_LR'] = pitch_bb_lr.astype(int)

## - W
pitch_w_lr = pitching[['W_LR']]
#scaled from 0 to 27
min = 0
max = 27
pitch_w_lr * min + (max - min)

pitching['W_LR'] = pitch_w_lr.astype(int)

## - SV
pitch_sv_lr = pitching[['SV_LR']]
#scaled from 0 to 62
min = 0
max = 62
pitch_sv_lr * min + (max - min)

pitching['SV_LR'] = pitch_sv_lr.astype(int)

pitching['Nxt_SO'] = pitching['Nxt_SO'].astype(int)
pitching['Nxt_W'] = pitching['Nxt_W'].astype(int)
pitching['Nxt_BB'] = pitching['Nxt_BB'].astype(int)
pitching['Nxt_SV'] = pitching['Nxt_SV'].astype(int)

In [124]:
pitching.insert(len(pitching.columns)-1, 'Nxt_ERA', pitching.pop('Nxt_ERA'))
pitching.insert(len(pitching.columns)-1, 'ERA_DTR', pitching.pop('ERA_DTR'))
pitching.insert(len(pitching.columns)-1, 'ERA_LR', pitching.pop('ERA_LR'))

pitching.insert(len(pitching.columns)-1, 'Nxt_SO', pitching.pop('Nxt_SO'))
pitching.insert(len(pitching.columns)-1, 'SO_DTR', pitching.pop('SO_DTR'))
pitching.insert(len(pitching.columns)-1, 'SO_LR', pitching.pop('SO_LR'))

pitching.insert(len(pitching.columns)-1, 'Nxt_WHIP', pitching.pop('Nxt_WHIP'))
pitching.insert(len(pitching.columns)-1, 'WHIP_DTR', pitching.pop('WHIP_DTR'))
pitching.insert(len(pitching.columns)-1, 'WHIP_LR', pitching.pop('WHIP_LR'))

pitching.insert(len(pitching.columns)-1, 'Nxt_BB', pitching.pop('Nxt_BB'))
pitching.insert(len(pitching.columns)-1, 'BB_DTR', pitching.pop('BB_DTR'))
pitching.insert(len(pitching.columns)-1, 'BB_LR', pitching.pop('BB_LR'))

pitching.insert(len(pitching.columns)-1, 'Nxt_W', pitching.pop('Nxt_W'))
pitching.insert(len(pitching.columns)-1, 'W_DTR', pitching.pop('W_DTR'))
pitching.insert(len(pitching.columns)-1, 'W_LR', pitching.pop('W_LR'))

pitching.insert(len(pitching.columns)-1, 'Nxt_SV', pitching.pop('Nxt_SV'))
pitching.insert(len(pitching.columns)-1, 'SV_DTR', pitching.pop('SV_DTR'))
pitching.insert(len(pitching.columns)-1, 'SV_LR', pitching.pop('SV_LR'))

pitching.head()

,Name,Age,Tm,W,L,W-L%,ERA,G,GS,GF,CG,SHO,SV,IP,H,R,ER,HR,BB,IBB,SO,HBP,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Year,Nxt_ERA,ERA_DTR,ERA_LR,Nxt_SO,SO_DTR,SO_LR,Nxt_WHIP,WHIP_DTR,WHIP_LR,Nxt_BB,BB_DTR,BB_LR,Nxt_W,W_DTR,W_LR,Nxt_SV,SV_DTR,SV_LR
0,A.J. Achter,0.233333,MIN,0.037037,0.000000,1.000,0.110236,0.019802,0.000000,0.011905,0.000000,0.0,0.0,0.026652,0.034211,0.036364,0.020408,0.04,0.014706,0.000000,0.013055,0.000000,0.0000,0.000000,0.026161,0.077377,0.287988,0.144087,0.303125,0.137931,0.046296,0.220430,0.0668,2014,6.75,5.11,4.97,14,32,39,1.350,1.48,1.494,6,18,21,0,2,3,0,0,0
1,A.J. Achter,0.266667,MIN,0.000000,0.045455,0.000,0.240720,0.059406,0.000000,0.047619,0.000000,0.0,0.0,0.032249,0.028947,0.054545,0.061224,0.08,0.029412,0.041667,0.036554,0.000000,0.0000,0.000000,0.032047,0.032424,0.330439,0.121699,0.196875,0.232759,0.075926,0.510753,0.0932,2015,3.11,5.12,4.75,14,33,37,1.460,1.46,1.464,12,17,19,1,2,2,0,0,1
3,A.J. Burnett,0.133333,FLA,0.148148,0.090909,0.667,0.118110,0.019802,0.142857,0.000000,0.000000,0.0,0.0,0.106876,0.094737,0.133333,0.102041,0.06,0.122549,0.083333,0.086162,0.000000,0.0000,0.000000,0.113146,0.078113,0.234925,0.138921,0.196875,0.060345,0.100000,0.387097,0.0528,1999,4.79,4.54,4.85,57,61,63,1.500,1.54,1.502,44,36,39,3,5,5,0,0,1
4,A.J. Burnett,0.166667,FLA,0.111111,0.318182,0.300,0.167229,0.079208,0.265306,0.000000,0.000000,0.0,0.0,0.216418,0.207895,0.272727,0.292517,0.16,0.215686,0.125000,0.148825,0.095238,0.0000,0.076923,0.232178,0.054532,0.253256,0.138921,0.215625,0.077586,0.088889,0.333333,0.0520,2000,4.05,4.58,5.00,128,60,70,1.315,1.50,1.496,83,36,43,11,6,5,0,0,0
5,A.J. Burnett,0.200000,FLA,0.407407,0.545455,0.478,0.139483,0.217822,0.551020,0.000000,0.066667,0.1,0.0,0.458689,0.378947,0.490909,0.523810,0.40,0.406863,0.125000,0.334204,0.333333,0.0625,0.269231,0.473512,0.064112,0.250844,0.117681,0.178125,0.086207,0.079630,0.354839,0.0616,2001,3.30,4.59,4.74,203,109,118,1.189,1.47,1.441,90,71,68,12,9,9,0,0,0


In [125]:
pitching

,Name,Age,Tm,W,L,W-L%,ERA,G,GS,GF,CG,SHO,SV,IP,H,R,ER,HR,BB,IBB,SO,HBP,BK,WP,BF,ERA+,FIP,WHIP,H9,HR9,BB9,SO9,SO/W,Year,Nxt_ERA,ERA_DTR,ERA_LR,Nxt_SO,SO_DTR,SO_LR,Nxt_WHIP,WHIP_DTR,WHIP_LR,Nxt_BB,BB_DTR,BB_LR,Nxt_W,W_DTR,W_LR,Nxt_SV,SV_DTR,SV_LR
0,A.J. Achter,0.233333,MIN,0.037037,0.000000,1.000,0.110236,0.019802,0.000000,0.011905,0.000000,0.0,0.0,0.026652,0.034211,0.036364,0.020408,0.04,0.014706,0.000000,0.013055,0.000000,0.0000,0.000000,0.026161,0.077377,0.287988,0.144087,0.303125,0.137931,0.046296,0.220430,0.0668,2014,6.75,5.11,4.97,14,32,39,1.350,1.48,1.494,6,18,21,0,2,3,0,0,0
1,A.J. Achter,0.266667,MIN,0.000000,0.045455,0.000,0.240720,0.059406,0.000000,0.047619,0.000000,0.0,0.0,0.032249,0.028947,0.054545,0.061224,0.08,0.029412,0.041667,0.036554,0.000000,0.0000,0.000000,0.032047,0.032424,0.330439,0.121699,0.196875,0.232759,0.075926,0.510753,0.0932,2015,3.11,5.12,4.75,14,33,37,1.460,1.46,1.464,12,17,19,1,2,2,0,0,1
3,A.J. Burnett,0.133333,FLA,0.148148,0.090909,0.667,0.118110,0.019802,0.142857,0.000000,0.000000,0.0,0.0,0.106876,0.094737,0.133333,0.102041,0.06,0.122549,0.083333,0.086162,0.000000,0.0000,0.000000,0.113146,0.078113,0.234925,0.138921,0.196875,0.060345,0.100000,0.387097,0.0528,1999,4.79,4.54,4.85,57,61,63,1.500,1.54,1.502,44,36,39,3,5,5,0,0,1
4,A.J. Burnett,0.166667,FLA,0.111111,0.318182,0.300,0.167229,0.079208,0.265306,0.000000,0.000000,0.0,0.0,0.216418,0.207895,0.272727,0.292517,0.16,0.215686,0.125000,0.148825,0.095238,0.0000,0.076923,0.232178,0.054532,0.253256,0.138921,0.215625,0.077586,0.088889,0.333333,0.0520,2000,4.05,4.58,5.00,128,60,70,1.315,1.50,1.496,83,36,43,11,6,5,0,0,0
5,A.J. Burnett,0.200000,FLA,0.407407,0.545455,0.478,0.139483,0.217822,0.551020,0.000000,0.066667,0.1,0.0,0.458689,0.378947,0.490909,0.523810,0.40,0.406863,0.125000,0.334204,0.333333,0.0625,0.269231,0.473512,0.064112,0.250844,0.117681,0.178125,0.086207,0.079630,0.354839,0.0616,2001,3.30,4.59,4.74,203,109,118,1.189,1.47,1.441,90,71,68,12,9,9,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23609,Zane Smith*,0.433333,PIT,0.296296,0.363636,0.500,0.102362,0.178218,0.448980,0.000000,0.133333,0.3,0.0,0.373134,0.360526,0.333333,0.319728,0.16,0.093137,0.125000,0.146214,0.095238,0.0000,0.000000,0.364290,0.070744,0.180415,0.094489,0.218750,0.043103,0.022222,0.193548,0.1180,1992,4.55,3.49,3.97,32,59,62,1.434,1.23,1.317,22,50,31,3,9,7,0,0,0
23610,Zane Smith*,0.466667,PIT,0.111111,0.318182,0.300,0.158230,0.089109,0.285714,0.000000,0.033333,0.0,0.0,0.218550,0.252632,0.254545,0.278912,0.10,0.107843,0.125000,0.083551,0.000000,0.0000,0.076923,0.224984,0.051584,0.210806,0.131343,0.271875,0.043103,0.044444,0.188172,0.0580,1993,3.27,4.14,4.52,57,59,41,1.248,1.42,1.447,34,32,25,10,5,4,0,0,0
23611,Zane Smith*,0.500000,PIT,0.370370,0.363636,0.556,0.110236,0.198020,0.489796,0.000000,0.066667,0.1,0.0,0.415778,0.423684,0.400000,0.380952,0.36,0.166667,0.291667,0.148825,0.000000,0.0000,0.076923,0.415958,0.084009,0.246985,0.109989,0.234375,0.086207,0.035185,0.177419,0.0672,1994,5.61,4.59,4.67,47,45,45,1.509,1.42,1.379,23,36,27,8,9,7,0,0,0
23612,Zane Smith*,0.533333,BOS,0.296296,0.363636,0.500,0.197975,0.188119,0.428571,0.000000,0.000000,0.0,0.0,0.291045,0.376316,0.466667,0.462585,0.14,0.112745,0.041667,0.122715,0.047619,0.0625,0.000000,0.310661,0.050847,0.207429,0.139954,0.309375,0.051724,0.035185,0.204301,0.0816,1995,5.08,4.23,4.63,47,46,45,1.500,1.32,1.417,21,24,23,4,6,5,0,0,0


# Model Comparisons

In [126]:
print('Batting Model Comparisons:')
print('')
print('Decision Tree Best BA MSE:',bat_ba_dtreg)
print('Linear Regression BA MSE:',bat_ba_lreg)
print('')
print('Decision Tree Best RBI MSE:',bat_rbi_dtreg)
print('Linear Regression RBI MSE:',bat_rbi_lreg)
print('')
print('Decision Tree Best HR MSE:',bat_hr_dtreg)
print('Linear Regression HR MSE:',bat_hr_lreg)
print('')
print('Decision Tree Best BB MSE:',bat_bb_dtreg)
print('Linear Regression BB MSE:',bat_bb_lreg)
print('')
print('Decision Tree Best SO MSE:',bat_so_dtreg)
print('Linear Regression SO MSE:',bat_so_lreg)
print('')
print('Pitching Model Comparisons:')
print('')
print('Decision Tree Best ERA MSE:',pitch_era_dtreg)
print('Linear Regression ERA MSE:',pitch_era_lreg)
print('')
print('Decision Tree Best SO MSE:',pitch_so_dtreg)
print('Linear Regression SO MSE:',pitch_so_lreg)
print('')
print('Decision Tree Best WHIP MSE:',pitch_whip_dtreg)
print('Linear Regression WHIP MSE:',pitch_whip_lreg)
print('')
print('Decision Tree Best BB MSE:',pitch_bb_dtreg)
print('Linear Regression BB MSE:',pitch_bb_lreg)
print('')
print('Decision Tree Best W MSE:',pitch_w_dtreg)
print('Linear Regression W MSE:',pitch_w_lreg)
print('')
print('Decision Tree Best SV MSE:',pitch_sv_dtreg)
print('Linear Regression SV MSE:',pitch_sv_lreg)

Batting Model Comparisons:

Decision Tree Best BA MSE: 0.00379
Linear Regression BA MSE: 0.00386

Decision Tree Best RBI MSE: 427.9941
Linear Regression RBI MSE: 394.91091

Decision Tree Best HR MSE: 41.79824
Linear Regression HR MSE: 38.55819

Decision Tree Best BB MSE: 248.42436
Linear Regression BB MSE: 230.07081

Decision Tree Best SO MSE: 707.86623
Linear Regression SO MSE: 647.49604

Pitching Model Comparisons:

Decision Tree Best ERA MSE: 3.64208
Linear Regression ERA MSE: 3.55128

Decision Tree Best SO MSE: 1402.50113
Linear Regression SO MSE: 1311.38268

Decision Tree Best WHIP MSE: 0.10075
Linear Regression WHIP MSE: 0.09735

Decision Tree Best BB MSE: 316.87505
Linear Regression BB MSE: 290.21958

Decision Tree Best W MSE: 14.37522
Linear Regression W MSE: 13.42998

Decision Tree Best SV MSE: 29.99157
Linear Regression SV MSE: 28.30667


In [127]:
batting_predicted = batting[['Name', 'Year',
                             'Nxt_BA','BA_LR',
                             'Nxt_RBI','RBI_LR',
                             'Nxt_HR','HR_LR',
                             'Nxt_BB','BB_LR',
                             'Nxt_SO','SO_LR',
                            ]].copy()

pitching_predicted = pitching[['Name', 'Year',
                               'Nxt_ERA','ERA_LR',
                               'Nxt_SO','SO_LR',
                               'Nxt_WHIP','WHIP_LR',
                               'Nxt_BB','BB_LR',
                               'Nxt_W','W_LR',
                               'Nxt_SV','SV_LR',
                              ]].copy()

batting_predicted.to_csv('predicted_batting.csv')
pitching_predicted.to_csv('predicted_pitching.csv')

In [128]:
batting_predicted

,Name,Year,Nxt_BA,BA_LR,Nxt_RBI,RBI_LR,Nxt_HR,HR_LR,Nxt_BB,BB_LR,Nxt_SO,SO_LR
0,A.J. Burnett,2001,0.105,0.150,3,5,1,1,5,4,28,27
1,A.J. Burnett,2002,0.138,0.167,1,9,0,2,0,8,11,31
2,A.J. Burnett,2004,0.147,0.155,2,3,1,0,1,2,34,15
3,A.J. Burnett,2005,0.063,0.168,2,6,0,1,3,4,36,33
4,A.J. Burnett,2012,0.068,0.138,2,-1,0,0,2,0,33,23
...,...,...,...,...,...,...,...,...,...,...,...,...
31688,Ángel Pagán#,2011,0.288,0.274,56,48,8,8,48,42,97,59
31689,Ángel Pagán#,2012,0.282,0.291,30,56,5,9,23,46,36,83
31690,Ángel Pagán#,2013,0.300,0.259,27,35,3,6,25,29,53,44
31691,Ángel Pagán#,2014,0.262,0.267,37,33,3,4,32,27,93,51


In [129]:
pitching_predicted

,Name,Year,Nxt_ERA,ERA_LR,Nxt_SO,SO_LR,Nxt_WHIP,WHIP_LR,Nxt_BB,BB_LR,Nxt_W,W_LR,Nxt_SV,SV_LR
0,A.J. Achter,2014,6.75,4.97,14,39,1.350,1.494,6,21,0,3,0,0
1,A.J. Achter,2015,3.11,4.75,14,37,1.460,1.464,12,19,1,2,0,1
3,A.J. Burnett,1999,4.79,4.85,57,63,1.500,1.502,44,39,3,5,0,1
4,A.J. Burnett,2000,4.05,5.00,128,70,1.315,1.496,83,43,11,5,0,0
5,A.J. Burnett,2001,3.30,4.74,203,118,1.189,1.441,90,68,12,9,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23609,Zane Smith*,1992,4.55,3.97,32,62,1.434,1.317,22,31,3,7,0,0
23610,Zane Smith*,1993,3.27,4.52,57,41,1.248,1.447,34,25,10,4,0,0
23611,Zane Smith*,1994,5.61,4.67,47,45,1.509,1.379,23,27,8,7,0,0
23612,Zane Smith*,1995,5.08,4.63,47,45,1.500,1.417,21,23,4,5,0,0


# NOTES

In [130]:
#####
# - NOTES
#####

In [131]:
#####
# - NOTES
#####